# The Attention Economy of Global Conflict

## Project Summary

**Project:** Forgotten Wars: Quantifying the Attention Hierarchy of Global Conflict  
**Author:** La'Tahvia Williams  
**Course:** DATA 23700 - Data Science for Human Rights | Autumn 2025  
**Date:** December 2025

### Research Question
**How strongly do documented conflict events correlate with Google search volume for conflict-related terms, and how does this relationship vary across crises?**

This investigation tests whether public attention follows humanitarian severity or operates according to other factors like Western political salience, geographic proximity, or media framing.

### Core Finding
**ATTENTION = f(Western Political Salience) NOT f(Human Suffering)**

Analysis of 884,216 conflict events from 2020-2025 across 11 countries reveals that correlation between documented violence and public search interest ranges from r = 0.77 (Palestine-Gaza war) to r = 0.03 (Yemen internal conflict), a 25-fold difference that has nothing to do with death toll, geographic distance, or media access.

**Key Evidence:**
- Yemen's 52,235 deaths generate r = 0.03 correlation
- USA's 496 deaths generate r = 0.75 correlation
- This represents a 416:1 death ratio with inverse attention patterns

### Methodology
- **Data Sources:** ACLED conflict database + Google Trends search data
- **Scope:** 11 countries, 2020-2025, 44 calibrated search terms
- **Analysis:** Pearson correlation coefficients, temporal decay analysis, pattern classification
- **Visualization:** 10 comprehensive charts using Altair

### Implications
This research demonstrates that attention allocation follows predictable patterns based on Western political priorities rather than humanitarian need, creating "forgotten wars" that remain systematically underfunded despite ongoing violence.

## 1. Setup & Data Loading

In [10]:
# My setup
import pandas as pd
import numpy as np
import altair as alt
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

### ACLED Data Integration

Prepare Data: ACLED Data

In [11]:
# Combine all regional datasets

regions = {
    "Africa": "ACLED Data/Africa_aggregated_data_up_to-2025-10-18.csv",
    "Asia-Pacific": "ACLED Data/Asia-Pacific_aggregated_data_up_to-2025-10-18.csv",
    "Europe-Central-Asia": "ACLED Data/Europe-Central-Asia_aggregated_data_up_to-2025-10-18.csv",
    "Latin-America-the-Caribbean": "ACLED Data/Latin-America-the-Caribbean_aggregated_data_up_to-2025-10-18.csv",
    "Middle-East": "ACLED Data/Middle-East_aggregated_data_up_to-2025-10-18.csv",
    "US-and-Canada": "ACLED Data/US-and-Canada_aggregated_data_up_to-2025-10-18.csv"
}

# Load all datasets
acleds = {}
for region_name, file_path in regions.items():
    acleds[region_name] = pd.read_csv(file_path)
    print(f"Loaded {region_name}: {len(acleds[region_name])} rows")

# Combine into single dataframe
acled = pd.concat(acleds.values(), ignore_index=True)
print(f"Combined dataset: {len(acled)} total rows")

Loaded Africa: 255889 rows
Loaded Asia-Pacific: 198169 rows
Loaded Europe-Central-Asia: 110481 rows
Loaded Latin-America-the-Caribbean: 161259 rows
Loaded Middle-East: 137579 rows
Loaded US-and-Canada: 20839 rows
Combined dataset: 884216 total rows


## 2. Data Preprocessing

**Data Preprocessing**

In [12]:
# Convert WEEK to datetime
acled['WEEK'] = pd.to_datetime(acled['WEEK'])
acled['YEAR'] = acled['WEEK'].dt.year
acled['MONTH'] = acled['WEEK'].dt.month
acled['YEAR_MONTH'] = acled['WEEK'].dt.to_period('M')

# Ensure numeric columns
acled['EVENTS'] = pd.to_numeric(acled['EVENTS'], errors='coerce').fillna(0)
acled['FATALITIES'] = pd.to_numeric(acled['FATALITIES'], errors='coerce').fillna(0)
acled['POPULATION_EXPOSURE'] = pd.to_numeric(acled['POPULATION_EXPOSURE'], errors='coerce').fillna(0)

acled

WEEK           REGION        COUNTRY   ADMIN1  \
0      2004-10-23  Northern Africa        Algeria    Adrar   
1      2005-04-23  Northern Africa        Algeria    Adrar   
2      2005-06-25  Northern Africa        Algeria    Adrar   
3      2008-12-13  Northern Africa        Algeria    Adrar   
4      2009-04-18  Northern Africa        Algeria    Adrar   
...           ...              ...            ...      ...   
884211 2025-01-25    North America  United States  Wyoming   
884212 2025-04-05    North America  United States  Wyoming   
884213 2025-04-19    North America  United States  Wyoming   
884214 2025-05-03    North America  United States  Wyoming   
884215 2025-10-04    North America  United States  Wyoming   

                    EVENT_TYPE SUB_EVENT_TYPE  EVENTS  FATALITIES  \
0                      Battles    Armed clash       1           2   
1                      Battles    Armed clash       1           0   
2                      Battles    Armed clash       1          14   
3                      Battles    Armed clash       1           3   
4                      Battles    Armed clash       1           2   
...                        ...            ...     ...         ...   
884211  Strategic developments          Other       1           0   
884212  Strategic developments          Other       1           0   
884213  Strategic developments          Other       1           0   
884214  Strategic developments          Other       1           0   
884215  Strategic developments          Other       1           0   

        POPULATION_EXPOSURE           DISORDER_TYPE      ID  \
0                       0.0      Political violence    47.0   
1                       0.0      Political violence    47.0   
2                       0.0      Political violence    47.0   
3                       0.0      Political violence    47.0   
4                       0.0      Political violence    47.0   
...                     ...                     ...     ...   
884211                  0.0  Strategic developments  3533.0   
884212                  0.0  Strategic developments  3533.0   
884213                  0.0  Strategic developments  3533.0   
884214                  0.0  Strategic developments  3533.0   
884215                  0.0  Strategic developments  3533.0   

        CENTROID_LATITUDE  CENTROID_LONGITUDE  YEAR  MONTH YEAR_MONTH  
0                 26.4839             -1.3880  2004     10    2004-10  
1                 26.4839             -1.3880  2005      4    2005-04  
2                 26.4839             -1.3880  2005      6    2005-06  
3                 26.4839             -1.3880  2008     12    2008-12  
4                 26.4839             -1.3880  2009      4    2009-04  
...                   ...                 ...   ...    ...        ...  
884211            42.9879           -107.5485  2025      1    2025-01  
884212            42.9879           -107.5485  2025      4    2025-04  
884213            42.9879           -107.5485  2025      4    2025-04  
884214            42.9879           -107.5485  2025      5    2025-05  
884215            42.9879           -107.5485  2025     10    2025-10  

[884216 rows x 16 columns]

In [13]:
# Summary Stats
total_events = acled['EVENTS'].sum()
total_fatalities = acled['FATALITIES'].sum()
unique_countries = acled['COUNTRY'].nunique()
unique_regions = acled['REGION'].nunique()
time_range = f"{acled['WEEK'].min().date()} to {acled['WEEK'].max().date()}"
unique_event_types = acled['EVENT_TYPE'].nunique()

# print stats
print(f"Total Events: {total_events:,.0f}")
print(f"Total Fatalities: {total_fatalities:,.0f}")
print(f"Unique Countries: {unique_countries}")
print(f"Unique Regions: {unique_regions}")
print(f"Time Range: {time_range}")
print(f"Unique Event Types: {unique_event_types}")

Total Events: 2,772,023
Total Fatalities: 2,399,712
Unique Countries: 238
Unique Regions: 17
Time Range: 1996-12-28 to 2025-10-18
Unique Event Types: 6


In [14]:
# Summary stats table for regions
summary_by_region = acled.groupby('REGION').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum',
    'COUNTRY': 'nunique',
    'EVENT_TYPE': 'nunique'
}).round(0).reset_index()
summary_by_region.columns = ['Region', 'Total Events', 'Total Fatalities', 'Countries', 'Event Types']
summary_by_region = summary_by_region.sort_values('Total Events', ascending=False)

display(summary_by_region)

Region  Total Events  Total Fatalities  Countries  \
8                 Middle East        555682            513190         15   
6                      Europe        529764            225144         53   
13                 South Asia        370430             75712          7   
12              South America        238377             93066         14   
9               North America        212065             66217          5   
14             Southeast Asia        156422            122488         11   
5              Eastern Africa        141703            351946         19   
2   Caucasus and Central Asia        109242            212080          9   
10            Northern Africa        101877            195428          6   
16             Western Africa        100437            220716         16   
4                   East Asia         82592               248          6   
7               Middle Africa         72132            284241          9   
15            Southern Africa         34681              5170          8   
3             Central America         34674             15055          7   
1                   Caribbean         24490             17666         28   
11                    Oceania          7446              1345         24   
0                  Antarctica             9                 0          1   

    Event Types  
8             6  
6             6  
13            6  
12            6  
9             6  
14            6  
5             6  
2             6  
10            6  
16            6  
4             6  
7             6  
15            6  
3             6  
1             6  
11            6  
0             1

### Data Aggregation

**Data aggr for visualzations**

In [15]:
# Top 15 countries by events
top_countries = acled.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index().sort_values('EVENTS', ascending=False).head(15)

# Monthly trend
monthly_trend = acled.groupby(acled['WEEK'].dt.to_period('M')).agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
monthly_trend['WEEK'] = monthly_trend['WEEK'].dt.to_timestamp()

# Yearly trend
yearly_trend = acled.groupby('YEAR').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

# Top event types
top_event_types = acled.groupby('EVENT_TYPE')['EVENTS'].sum().reset_index().sort_values('EVENTS', ascending=False)

# Country-level aggregation for choropleth
country_stats = acled.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum',
    'POPULATION_EXPOSURE': 'sum'
}).reset_index()
country_stats['FATALITY_RATE'] = country_stats['FATALITIES'] / (country_stats['EVENTS'] + 1)

# Geographic scatter (by country centroid)
map_data = acled.groupby(['COUNTRY', 'CENTROID_LATITUDE', 'CENTROID_LONGITUDE']).agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"15 Top Countries")
display(top_countries)
print(f"Monthly Trends")
display(monthly_trend)
print(f"Yearly Trends")
display(yearly_trend)
print(f"Events by region and month")
print(f"Top event types")
display(top_event_types)
print(f"Country-level aggregation for choropleth")
display(country_stats)
print(f"Geographic scatter (by country centroid)")
display(map_data)


15 Top Countries


COUNTRY  EVENTS  FATALITIES
222        Ukraine  269387      217249
97           India  195161       13359
207          Syria  145292      141446
135         Mexico  111153       65708
160       Pakistan  108435       53281
234          Yemen  107829      165446
31          Brazil  102820       54864
144        Myanmar   96504       98567
225  United States   91948         496
162      Palestine   82404       71263
0      Afghanistan   68519      203244
100           Iraq   66004      107355
197        Somalia   52623       89449
217         Turkey   46596        9700
175         Russia   46133        7220

Monthly Trends


WEEK  EVENTS  FATALITIES
0   1996-12-01      27          72
1   1997-01-01     214        3369
2   1997-02-01     143        1753
3   1997-03-01     265        1612
4   1997-04-01     222        2017
..         ...     ...         ...
342 2025-06-01   32602       18892
343 2025-07-01   30655       17295
344 2025-08-01   37324       19951
345 2025-09-01   33175       15809
346 2025-10-01   22626        9874

[347 rows x 3 columns]

Yearly Trends


...

EVENT_TYPE   EVENTS
2                    Protests  1130708
1  Explosions/Remote violence   500835
0                     Battles   460185
5  Violence against civilians   310934
4      Strategic developments   191609
3                       Riots   177752

Country-level aggregation for choropleth


COUNTRY  EVENTS  FATALITIES  POPULATION_EXPOSURE  \
0              Afghanistan   68519      203244          909685303.0   
1    Akrotiri and Dhekelia      15           0               4469.0   
2                  Albania    1384           4           24590278.0   
3                  Algeria   12799       13801           65263229.0   
4           American Samoa       7           0              12324.0   
..                     ...     ...         ...                  ...   
233      Wallis and Futuna      27           0               7560.0   
234                  Yemen  107829      165446          231902879.0   
235                 Zambia    1578         355           11602267.0   
236               Zimbabwe    6733         635           12117596.0   
237               eSwatini     842         148            3638987.0   

     FATALITY_RATE  
0         2.966200  
1         0.000000  
2         0.002888  
3         1.078203  
4         0.000000  
..             ...  
233       0.000000  
234       1.534323  
235       0.224826  
236       0.094298  
237       0.175563  

[238 rows x 5 columns]

Geographic scatter (by country centroid)


COUNTRY  CENTROID_LATITUDE  CENTROID_LONGITUDE  EVENTS  FATALITIES
0     Afghanistan            30.8409             62.4753     697        1379
1     Afghanistan            31.0757             65.6926    4948       18939
2     Afghanistan            31.3969             64.0428    6243       17785
3     Afghanistan            32.2774             67.0193    2211        6636
4     Afghanistan            32.4966             62.4100    1988        7441
...           ...                ...                 ...     ...         ...
3526     Zimbabwe           -16.6813             31.6098     554          79
3527     eSwatini           -27.0244             31.3137     104          27
3528     eSwatini           -26.5546             31.8619     122          22
3529     eSwatini           -26.5264             31.1888     306          65
3530     eSwatini           -26.1033             31.3640     310          34

[3531 rows x 5 columns]

## GEOJSON

In [16]:
alt.data_transformers.disable_max_rows()

# Geojson
# import requests

WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/datasets/geo-countries/master/data/countries.geojson"

geo = requests.get(WORLD_GEOJSON_URL).json()
geo_countries = [f['properties']['name'] for f in geo['features']]

# Fix country name mismatches
fix_names = {
    'Democratic Republic of Congo': 'Democratic Republic of the Congo',
    'Republic of Congo': 'Republic of the Congo',
    'Tanzania': 'United Republic of Tanzania',
    'eSwatini': 'Swaziland',
    'United States': 'United States of America',
    'Palestine': 'Palestine, State of',
    'Vatican City': 'Holy See',
    'Serbia': 'Republic of Serbia',
    'Cape Verde': 'Cabo Verde',
    'North Macedonia': 'Macedonia',
    'Iran': 'Iran (Islamic Republic of)',
    'Syria': 'Syrian Arab Republic',
    'Russia': 'Russian Federation'
}

country_stats['COUNTRY'] = country_stats['COUNTRY'].replace(fix_names)

# Remove Bermuda (covers entire map, skews visualization)
country_stats = country_stats[country_stats['COUNTRY'] != 'Bermuda']
print("Removed Bermuda from dataset (causes map distortion).")

# Keep only countries found in GeoJSON
country_stats = country_stats[country_stats['COUNTRY'].isin(geo_countries)].reset_index(drop=True)
print(f"Matched {len(country_stats)} countries to GeoJSON features.")




NameError: name 'requests' is not defined

## EA VISUALIZATIONS

In [ ]:
# Top 15 countries by events bar graph
chart1 = alt.Chart(top_countries.reset_index(drop=True)).mark_bar().encode(
    y=alt.Y('COUNTRY:N', sort='-x', title='Country'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='reds'), title='Fatalities'),
    tooltip=['COUNTRY', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=700, height=400,
    title='Top 15 Countries by Total Conflict Events'
)

chart1

alt.Chart(...)

In [ ]:
# Monthy trends with fatalites

chart2_events = alt.Chart(monthly_trend).mark_line(point=True, color='steelblue', size=2).encode(
    x=alt.X('WEEK:T', title='Date'),
    y=alt.Y('EVENTS:Q', title='Events', axis=alt.Axis(labelColor='steelblue', titleColor='steelblue')),
    tooltip=['WEEK:T', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=900, height=350,
    title='Monthly Trend: Conflict Events Over Time'
).interactive()

chart2_fatalities = alt.Chart(monthly_trend).mark_line(point=True, color='darkred', size=2).encode(
    x=alt.X('WEEK:T', title='Date'),
    y=alt.Y('FATALITIES:Q', title='Fatalities', axis=alt.Axis(orient='right', labelColor='darkred', titleColor='darkred')),
    tooltip=['WEEK:T', 'EVENTS:Q', 'FATALITIES:Q']
)

monthly_combined = alt.layer(chart2_events, chart2_fatalities).resolve_scale(y='independent')

monthly_combined

alt.LayerChart(...)

In [ ]:
# Yearly Trends

chart3 = alt.Chart(yearly_trend).mark_bar(color='orange').encode(
    x=alt.X('YEAR:O', title='Year'),
    y=alt.Y('EVENTS:Q', title='Total Events'),
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='reds'), title='Fatalities'),
    tooltip=['YEAR', 'EVENTS:Q', 'FATALITIES:Q']
).properties(
    width=600, height=300,
    title='Annual Conflict Events: Yearly Aggregation'
)

chart3

alt.Chart(...)

In [ ]:
# 4. REGIONAL CONCENTRATION

regional_events = acled.groupby('REGION')['EVENTS'].sum().reset_index().sort_values('EVENTS', ascending=False)
chart4 = alt.Chart(regional_events).mark_bar().encode(
    y=alt.Y('REGION:N', sort='-x', title='Region'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    color=alt.value('teal'),
    tooltip=['REGION', 'EVENTS:Q']
).properties(
    width=600, height=250,
    title='Regional Concentration: Total Events by Region'
)

chart4

alt.Chart(...)

In [ ]:
# 5. EVENT TYPE DISTRIBUTION

chart5 = alt.Chart(top_event_types.head(10)).mark_bar(color='teal').encode(
    y=alt.Y('EVENT_TYPE:N', sort='-x', title='Event Type'),
    x=alt.X('EVENTS:Q', title='Total Events'),
    tooltip=['EVENT_TYPE', 'EVENTS:Q']
).properties(
    width=700, height=300,
    title='Top 10 Event Types by Frequency'
)

chart5

alt.Chart(...)

In [ ]:
country_stats

COUNTRY  EVENTS  FATALITIES  POPULATION_EXPOSURE  \
0              Afghanistan   68519      203244          909685303.0   
1    Akrotiri and Dhekelia      15           0               4469.0   
2                  Albania    1384           4           24590278.0   
3                  Algeria   12799       13801           65263229.0   
4           American Samoa       7           0              12324.0   
..                     ...     ...         ...                  ...   
233      Wallis and Futuna      27           0               7560.0   
234                  Yemen  107829      165446          231902879.0   
235                 Zambia    1578         355           11602267.0   
236               Zimbabwe    6733         635           12117596.0   
237               eSwatini     842         148            3638987.0   

     FATALITY_RATE  
0         2.966200  
1         0.000000  
2         0.002888  
3         1.078203  
4         0.000000  
..             ...  
233       0.000000  
234       1.534323  
235       0.224826  
236       0.094298  
237       0.175563  

[238 rows x 5 columns]

In [ ]:
# Interactive map

WORLD_GEOJSON_URL = "https://raw.githubusercontent.com/johan/world.geo.json/master/countries.geo.json"

# Define metrics to display
metrics = ['EVENTS', 'FATALITIES', 'FATALITY_RATE']

# Create dropdown selector
metric_sel = alt.selection_point(
    fields=['Metric'],
    bind=alt.binding_select(options=metrics, name='View Metric: '),
    value='EVENTS'
)

# Background (base countries layer in light gray)
base = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).encode(
    color=alt.value('#eaeaea')
).project('mercator')

# Interactive choropleth layer
choropleth = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data=country_stats, key='COUNTRY', fields=['EVENTS', 'FATALITIES', 'FATALITY_RATE'])
).transform_fold(
    metrics,
    as_=['Metric', 'Value']
).transform_filter(
    metric_sel
).encode(
    color=alt.Color(
        'Value:Q',
        scale=alt.Scale(scheme='reds'),
        title='Value'
    ),
    tooltip=[
        alt.Tooltip('properties.name:N', title='Country'),
        alt.Tooltip('Metric:N', title='Metric'),
        alt.Tooltip('Value:Q', title='Value', format='.2f')
    ]
).project(
    type='mercator'
).add_params(
    metric_sel
)

In [ ]:
# Combine base + interactive layer
chart8 = (base + choropleth).properties(
    width=1200, height=700,
    title='Interactive Choropleth: Switch between EVENTS / FATALITIES / FATALITY_RATE'
)

chart8

alt.LayerChart(...)

In [ ]:
chart9 = alt.Chart(
    alt.Data(url=WORLD_GEOJSON_URL, format={'type': 'json', 'property': 'features'})
).mark_geoshape(stroke='white', strokeWidth=0.25).transform_lookup(
    lookup='properties.name',
    from_=alt.LookupData(data=country_stats, key='COUNTRY', fields=['EVENTS', 'FATALITIES'])
).encode(
    color=alt.Color('FATALITIES:Q', scale=alt.Scale(scheme='purples'), title='Total Fatalities'),
    tooltip=[
        alt.Tooltip('properties.name:N', title='Country'),
        alt.Tooltip('FATALITIES:Q', title='Fatalities', format=','),
        alt.Tooltip('EVENTS:Q', title='Events', format=',')
    ]
).project(
    type='mercator'
).properties(
    width=1200, height=700,
    title='Map 3: Fatalities by Country (Static View)'
)

chart9

alt.Chart(...)

In [ ]:
# Fatality intensity by country graph

fatality_countries = acled.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
fatality_countries['INTENSITY'] = fatality_countries['FATALITIES'] / (fatality_countries['EVENTS'] + 1)
fatality_countries = fatality_countries.sort_values('FATALITIES', ascending=False).head(12)

chart11 = alt.Chart(fatality_countries.reset_index(drop=True)).mark_bar().encode(
    y=alt.Y('COUNTRY:N', sort='-x', title='Country'),
    x=alt.X('FATALITIES:Q', title='Total Fatalities'),
    color=alt.Color('INTENSITY:Q', scale=alt.Scale(scheme='reds'), title='Fatality Intensity'),
    tooltip=['COUNTRY', 'FATALITIES:Q', 'EVENTS:Q', 'INTENSITY:Q']
).properties(
    width=700, height=350,
    title='Top 12 Countries by Total Fatalities (Color: Fatality Intensity)'
)

chart11

alt.Chart(...)

## Top 20 Countries (All Years)

In [ ]:

# Top 20 Countries (All Years)

top_countries_overall = acled.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index().sort_values('EVENTS', ascending=False).head(20)

print("Top 20 Countries (All Years)")
display(top_countries_overall.to_string(index=False))

# Top Countries by year

print("Top Countries by year")

for year in sorted(acled['YEAR'].unique()):
    top_by_year = acled[acled['YEAR'] == year].groupby('COUNTRY').agg({
        'EVENTS': 'sum',
        'FATALITIES': 'sum'
    }).reset_index().sort_values('EVENTS', ascending=False).head(5)
    
    print(f"\n{year}:")
    print(top_by_year.to_string(index=False))

# Top Countries from 2020-2025

acled_2020_2025 = acled[acled['YEAR'].between(2020, 2025)]

top_countries_2020_2025 = acled_2020_2025.groupby('COUNTRY').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index().sort_values('EVENTS', ascending=False).head(20)

print("Top Countries from 2020-2025")
display(top_countries_2020_2025.to_string(index=False))

# Event type distribution for top 10 cuntries

top_10_countries = top_countries_2020_2025.head(10)['COUNTRY'].tolist()

print("Event type distribution for top 10 cuntries")

event_type_dist = acled_2020_2025[acled_2020_2025['COUNTRY'].isin(top_10_countries)].groupby(
    ['COUNTRY', 'EVENT_TYPE']
)['EVENTS'].sum().reset_index().pivot(
    index='COUNTRY', columns='EVENT_TYPE', values='EVENTS'
).fillna(0)

display(event_type_dist.to_string())

# Summary stats for google trends data

print("Summary stats for google trends data")

print(f"\nYour Google Trends data covers: 2020-01-01 to 2025-10-18")
print(f"Your ACLED data for this period: {acled_2020_2025['WEEK'].min().date()} to {acled_2020_2025['WEEK'].max().date()}")
print(f"\nTop 10 Priority Countries for Search Term Collection:")
print("-" * 80)

for idx, row in top_countries_2020_2025.head(10).iterrows():
    print(f"{idx+1:2d}. {row['COUNTRY']:25s} | Events: {row['EVENTS']:6.0f} | Fatalities: {row['FATALITIES']:6.0f}")


Top 20 Countries (All Years)


'                     COUNTRY  EVENTS  FATALITIES\n                     Ukraine  269387      217249\n                       India  195161       13359\n                       Syria  145292      141446\n                      Mexico  111153       65708\n                    Pakistan  108435       53281\n                       Yemen  107829      165446\n                      Brazil  102820       54864\n                     Myanmar   96504       98567\n               United States   91948         496\n                   Palestine   82404       71263\n                 Afghanistan   68519      203244\n                        Iraq   66004      107355\n                     Somalia   52623       89449\n                      Turkey   46596        9700\n                      Russia   46133        7220\n                 South Korea   44092           0\n                     Nigeria   43907      131658\n                      France   42782          51\nDemocratic Republic of Congo   38149       90794\

Top Countries by year

1996:
                 COUNTRY  EVENTS  FATALITIES
            Sierra Leone      13           0
                 Algeria       3           9
                  Angola       2          40
Central African Republic       2           4
                   Sudan       2           0

1997:
                     COUNTRY  EVENTS  FATALITIES
                Sierra Leone     959          40
                      Angola     259        5218
                       Sudan     252        5110
                     Burundi     219        4346
Democratic Republic of Congo     219        2291

1998:
                     COUNTRY  EVENTS  FATALITIES
                Sierra Leone    1689           0
                      Angola     519       52810
                      Uganda     314        1161
Democratic Republic of Congo     307        3376
                       Sudan     260        4046

1999:
                     COUNTRY  EVENTS  FATALITIES
                      Angola    1344       

'      COUNTRY  EVENTS  FATALITIES\n      Ukraine  237510      215941\n        India  125521        6660\nUnited States   91922         496\n      Myanmar   86872       89665\n       Mexico   84682       46812\n        Syria   72821       41178\n       Brazil   72011       41686\n    Palestine   68426       70366\n        Yemen   58708       52235\n     Pakistan   53662       13782\n         Iraq   42783       10317\n       France   42756          51\n       Russia   42290        7053\n  South Korea   34565           0\n       Turkey   32733        1001\n     Colombia   28643       11883\n      Lebanon   28421        4845\n      Nigeria   28001       57795\n        Italy   27815           2\n  Afghanistan   27339       81549'

Event type distribution for top 10 cuntries


'EVENT_TYPE     Battles  Explosions/Remote violence  Protests  Riots  Strategic developments  Violence against civilians\nCOUNTRY                                                                                                                \nBrazil           27076                         336     14188   3455                    2706                       24250\nIndia             3369                         829     98933  12529                    5906                        3955\nMexico            7665                         251     34938   3880                    5530                       32418\nMyanmar          20943                       22975     15386     85                   17329                       10154\nPakistan          3893                        2146     41641   1839                    1183                        2960\nPalestine         5437                       21719      3639  21445                   11194                        4992\nSyria            13376         

Summary stats for google trends data

Your Google Trends data covers: 2020-01-01 to 2025-10-18
Your ACLED data for this period: 2020-01-04 to 2025-10-18

Top 10 Priority Countries for Search Term Collection:
--------------------------------------------------------------------------------
223. Ukraine                   | Events: 237510 | Fatalities: 215941
98. India                     | Events: 125521 | Fatalities:   6660
226. United States             | Events:  91922 | Fatalities:    496
145. Myanmar                   | Events:  86872 | Fatalities:  89665
136. Mexico                    | Events:  84682 | Fatalities:  46812
208. Syria                     | Events:  72821 | Fatalities:  41178
32. Brazil                    | Events:  72011 | Fatalities:  41686
163. Palestine                 | Events:  68426 | Fatalities:  70366
235. Yemen                     | Events:  58708 | Fatalities:  52235
161. Pakistan                  | Events:  53662 | Fatalities:  13782


### Syria Analysis - Key Findings

**Correlation:** r = 0.16 (weak - near forgotten)  
**Deaths (2020-2025):** 41,178  
**Pattern:** Forgotten War - attention fully decoupled from violence  

**What This Means:**
Syria exemplifies the "forgotten war" pattern: despite 41,178 deaths between 2020-2025 and ongoing conflict, the civil war generates almost no public attention (r = 0.16). The weak correlation indicates that search interest operates almost independently of violence levels.

After dominating headlines in 2016-2018, Syria became normalized as a "permanent crisis." Media fatigue, shifting geopolitical priorities, and lack of clear Western intervention strategy led to complete attention collapse. This demonstrates how conflicts can become invisible despite continued humanitarian catastrophe—violence continues, but attention has moved elsewhere.

## INDIVIDUAL COUNTRY CHARTS

In [ ]:
# Convert WEEK to datetime
acled['WEEK'] = pd.to_datetime(acled['WEEK'])

# Filter to 2020-2025 period
acled_recent = acled[acled['WEEK'] >= '2020-01-01'].copy()

# Top 10 countries by events in this period
top_countries = ['Ukraine', 'India', 'United States', 'Myanmar', 'Mexico', 
                 'Syria', 'Brazil', 'Palestine', 'Yemen', 'Pakistan']

# Filter to top countries
acled_top = acled_recent[acled_recent['COUNTRY'].isin(top_countries)].copy()

# Aggregate by month and country
monthly_summary = acled_top.groupby([pd.Grouper(key='WEEK', freq='M'), 'COUNTRY']).agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

monthly_summary['WEEK'] = monthly_summary['WEEK'].dt.to_period('M').dt.to_timestamp()

# Create multi-line chart for EVENTS
events_chart = alt.Chart(monthly_summary).mark_line(point=True).encode(
    x=alt.X('WEEK:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('EVENTS:Q', 
            title='Total Events per Month',
            scale=alt.Scale(type='linear')),
    color=alt.Color('COUNTRY:N', 
                    title='Country',
                    scale=alt.Scale(scheme='category20')),
    tooltip=[
        alt.Tooltip('COUNTRY:N', title='Country'),
        alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
        alt.Tooltip('EVENTS:Q', title='Events', format=','),
        alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
    ]
).properties(
    width=1000,
    height=400,
    title='Monthly Events by Country (2020-2025)'
).interactive()

# Multi-line chart for fatalities
fatalities_chart = alt.Chart(monthly_summary).mark_line(point=True).encode(
    x=alt.X('WEEK:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('FATALITIES:Q', 
            title='Total Fatalities per Month',
            scale=alt.Scale(type='linear')),
    color=alt.Color('COUNTRY:N', 
                    title='Country',
                    scale=alt.Scale(scheme='category20')),
    tooltip=[
        alt.Tooltip('COUNTRY:N', title='Country'),
        alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
        alt.Tooltip('EVENTS:Q', title='Events', format=','),
        alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
    ]
).properties(
    width=1000,
    height=400,
    title='Monthly Fatalities by Country (2020-2025)'
).interactive()

# Combine charts vertically
combined_chart = alt.vconcat(events_chart, fatalities_chart)

# Display
combined_chart.save('acled_temporal_overview.html')
print("✓ Saved: acled_temporal_overview.html")

# Individual country charts to identify specific spikes
 
print("CREATING INDIVIDUAL COUNTRY CHARTS")
 

for country in top_countries:
    country_data = monthly_summary[monthly_summary['COUNTRY'] == country].copy()
    
    # Top 5 spike months for this country
    top_spikes = country_data.nlargest(5, 'EVENTS')[['WEEK', 'EVENTS', 'FATALITIES']]
    
    print(f"\n{country.upper()}")
     
    print("Top 5 Event Spikes:")
    for idx, row in top_spikes.iterrows():
        print(f"  {row['WEEK'].strftime('%B %Y')}: {row['EVENTS']:,} events, {row['FATALITIES']:,} fatalities")
    
    # Dual-axis chart for this country
    base = alt.Chart(country_data).encode(
        x=alt.X('WEEK:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    events_line = base.mark_line(color='steelblue', strokeWidth=2).encode(
        y=alt.Y('EVENTS:Q', 
                title='Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
        ]
    )
    
    events_points = base.mark_circle(color='steelblue', size=50).encode(
        y=alt.Y('EVENTS:Q'),
        tooltip=[
            alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
        ]
    )
    
    fatalities_line = base.mark_line(color='red', strokeWidth=2).encode(
        y=alt.Y('FATALITIES:Q', 
                title='Fatalities',
                axis=alt.Axis(titleColor='red')),
        tooltip=[
            alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
        ]
    )
    
    fatalities_points = base.mark_circle(color='red', size=50).encode(
        y=alt.Y('FATALITIES:Q'),
        tooltip=[
            alt.Tooltip('WEEK:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip('FATALITIES:Q', title='Fatalities', format=',')
        ]
    )
    
    country_chart = alt.layer(events_line, events_points, fatalities_line, fatalities_points).resolve_scale(
        y='independent'
    ).properties(
        width=1000,
        height=400,
        title=f'{country}: Monthly Events and Fatalities (2020-2025)'
    ).interactive()
    
    # Individual chart
    filename = f"acled_{country.lower().replace(' ', '_')}.html"
    country_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: acled_temporal_overview.html
CREATING INDIVIDUAL COUNTRY CHARTS

UKRAINE
Top 5 Event Spikes:
  August 2025: 7,577 events, 6,158 fatalities
  May 2025: 7,562 events, 7,907 fatalities
  March 2025: 6,957 events, 8,480 fatalities
  September 2025: 6,201 events, 4,487 fatalities
  July 2025: 6,091 events, 4,563 fatalities
✓ Saved: acled_ukraine.html

INDIA
Top 5 Event Spikes:
  August 2024: 3,421 events, 87 fatalities
  June 2024: 2,839 events, 141 fatalities
  July 2023: 2,657 events, 196 fatalities
  March 2025: 2,649 events, 140 fatalities
  September 2023: 2,625 events, 102 fatalities
✓ Saved: acled_india.html

UNITED STATES
Top 5 Event Spikes:
  June 2020: 4,736 events, 15 fatalities
  May 2020: 4,454 events, 19 fatalities
  June 2025: 2,766 events, 3 fatalities
  August 2020: 2,764 events, 12 fatalities
  April 2025: 2,482 events, 3 fatalities
✓ Saved: acled_united_states.html

MYANMAR
Top 5 Event Spikes:
  May 2021: 2,455 events, 971 fatalities
  February 2021: 2,147 events

In [ ]:
combined_chart

alt.VConcatChart(...)

### Ukraine Analysis - Key Findings

**Correlation:** r = 0.40 (moderate-strong)  
**Deaths (2020-2025):** 215,941  
**Pattern:** Sudden Shock with exponential decay  

**What This Means:**
Ukraine exemplifies the "sudden shock" pattern: Russia's February 24, 2022 invasion generated massive global attention (100% search interest), but attention decayed to just 5.5% by Year 3 despite ongoing violence claiming hundreds of lives weekly. This 94.5% decline demonstrates that even geopolitically salient conflicts face exponential attention decay with an approximately 3-month half-life.

The moderate-strong correlation (r = 0.40) reflects sustained attention during major escalations (invasion, counteroffensives) but also shows how attention becomes decoupled from humanitarian severity over time. Ukraine receives far more sustained attention than most conflicts due to Western political salience, but even this elevated attention follows predictable decay patterns.

## Ukraine Google Trends Data

In [ ]:
# Ukraine Google Trends Data

# Filter for Ukraine, 2020-2025
ukraine_acled = acled[
    (acled['COUNTRY'] == 'Ukraine') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level to match Google Trends
ukraine_acled['month'] = ukraine_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
ukraine_monthly = ukraine_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"\nACLED Data: {len(ukraine_monthly)} months")
print(f"Date range: {ukraine_monthly['month'].min()} to {ukraine_monthly['month'].max()}")
print(f"Total events: {ukraine_monthly['EVENTS'].sum():,}")
print(f"Total fatalities: {ukraine_monthly['FATALITIES'].sum():,}")



ACLED Data: 70 months
Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
Total events: 237,510
Total fatalities: 215,941


In [ ]:
# Load all Ukraine trends files
trends_files = {
    'Ukraine war': 'Ukraine CSVs/Ukraine_war.csv',
    'Ukraine news': 'Ukraine CSVs/Ukraine_news.csv',
    'Russia Ukraine': 'Ukraine CSVs/Russia_Ukraine.csv',
    'Russian invasion': 'Ukraine CSVs/Russian invasion.csv',
    'Ukraine invasion': 'Ukraine CSVs/Ukraine_invasion.csv',
    'Ukraine conflict': 'Ukraine CSVs/Ukraine_conflict.csv',
    'Kyiv attack': 'Ukraine CSVs/Kyiv_attack.csv',
    'Zelensky': 'Ukraine CSVs/Zelensky.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)  # Skip the "Category" row
        df.columns = ['month', 'value']
        
        # Convert month to datetime
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values (convert to 0.5)
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'])
        
        trends_data[name] = df
        print(f"✓ Loaded: {name:20s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"✗ Error loading {name}: {e}")

✓ Loaded: Ukraine war          - 70 months, max=100.0
✓ Loaded: Ukraine news         - 70 months, max=100.0
✓ Loaded: Russia Ukraine       - 70 months, max=100.0
✓ Loaded: Russian invasion     - 70 months, max=100.0
✓ Loaded: Ukraine invasion     - 70 months, max=100.0
✓ Loaded: Ukraine conflict     - 70 months, max=100.0
✓ Loaded: Kyiv attack          - 70 months, max=100
✓ Loaded: Zelensky             - 70 months, max=100.0


In [ ]:
# Merge with ACLED data
merged = ukraine_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"\n✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")


✓ Merged dataset: 70 months with 8 search terms


In [ ]:
merged

month  EVENTS  FATALITIES  Ukraine war  Ukraine news  Russia Ukraine  \
0  2020-01-01    1045          18          0.5           1.0             0.5   
1  2020-02-01    1530          14          0.5           0.5             0.5   
2  2020-03-01    1266          13          0.5           0.5             0.5   
3  2020-04-01    1059           8          0.5           0.5             0.5   
4  2020-05-01    1407          12          0.5           0.5             0.5   
..        ...     ...         ...          ...           ...             ...   
65 2025-06-01    6038        5918         14.0           6.0             3.0   
66 2025-07-01    6091        4563         11.0           5.0             2.0   
67 2025-08-01    7577        6158         22.0          12.0             6.0   
68 2025-09-01    6201        4487         18.0          14.0             4.0   
69 2025-10-01    4546        3680         15.0          13.0             3.0   

    Russian invasion  Ukraine invasion  Ukraine conflict  Kyiv attack  \
0                1.0               0.5               0.5            0   
1                1.0               0.5               0.5            0   
2                1.0               0.5               0.5            0   
3                1.0               0.5               0.5            0   
4                1.0               0.5               0.5            0   
..               ...               ...               ...          ...   
65               2.0               2.0               3.0           28   
66               2.0               2.0               2.0           28   
67               3.0               4.0               8.0          100   
68               3.0               3.0               6.0           82   
69               2.0               3.0               4.0           35   

    Zelensky  
0        1.0  
1        0.5  
2        0.5  
3        0.5  
4        0.5  
..       ...  
65       6.0  
66       5.0  
67      28.0  
68       7.0  
69       6.0  

[70 rows x 11 columns]

CALCULATE CORRELATIONS

In [ ]:
# Correlations

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        # Remove NaN values for correlation
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:  # Need sufficient data
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))


     Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
     Kyiv attack        0.398752            0.314699           70
     Ukraine war        0.395183            0.365977           70
        Zelensky        0.262073            0.324027           70
    Ukraine news        0.171388            0.171367           70
  Russia Ukraine       -0.002952            0.033917           70
Ukraine invasion       -0.014544            0.025269           70
Ukraine conflict       -0.019110            0.009118           70
Russian invasion       -0.021962            0.018594           70


 TIME-LAGGED CORRELATION ANALYSIS:

- Do searches LEAD or LAG events?

- Positive lag = Searches happen AFTER events (reactive)

- Negative lag = Searches happen BEFORE events (predictive)

In [ ]:
# Time-Lagged Correlation Anaysis

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            # Positive lag: shift events forward (searches lag behind events)
            corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
        else:
            # Negative lag: shift searches forward (searches lead events)
            corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")



Kyiv attack:
  Lag -3 months (searches LEAD  ): correlation = +0.268
  Lag -2 months (searches LEAD  ): correlation = +0.357
  Lag -1 months (searches LEAD  ): correlation = +0.390
  Lag +0 months (CONCURRENT     ): correlation = +0.399
  Lag +1 months (searches LAG   ): correlation = +0.390
  Lag +2 months (searches LAG   ): correlation = +0.357
  Lag +3 months (searches LAG   ): correlation = +0.268

 Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Ukraine war:
  Lag -3 months (searches LEAD  ): correlation = +0.376
  Lag -2 months (searches LEAD  ): correlation = +0.383
  Lag -1 months (searches LEAD  ): correlation = +0.388
  Lag +0 months (CONCURRENT     ): correlation = +0.395
  Lag +1 months (searches LAG   ): correlation = +0.388
  Lag +2 months (searches LAG   ): correlation = +0.383
  Lag +3 months (searches LAG   ): correlation = +0.376

 Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Zelensky:
  Lag -3 months (searches

## VISUALIZATIONS

In [ ]:
# Visualizations

# Normalize data
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] - merged['EVENTS'].min()) / (merged['EVENTS'].max() - merged['EVENTS'].min()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] - merged['FATALITIES'].min()) / (merged['FATALITIES'].max() - merged['FATALITIES'].min()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events (normalized)',
        'value': row['EVENTS_norm'],
        'type': 'ACLED'
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities (normalized)',
        'value': row['FATALITIES_norm'],
        'type': 'ACLED'
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends'
            })

plot_df = pd.DataFrame(plot_data)

# Multi-line chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 100])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Source',
                               scale=alt.Scale(domain=['ACLED', 'Google Trends'],
                                             range=[[1,0], [4,2]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Value', format='.1f')
    ]
).properties(
    width=1200,
    height=500,
    title='Ukraine: ACLED Events vs Google Search Interest (2020-2025)'
).interactive()

# Annotation for Russian invasion
invasion_date = pd.to_datetime('2022-02-01')
invasion_line = alt.Chart(pd.DataFrame({'x': [invasion_date]})).mark_rule(
    color='red',
    strokeWidth=2,
    strokeDash=[5, 5]
).encode(
    x='x:T'
)

invasion_text = alt.Chart(pd.DataFrame({
    'x': [invasion_date],
    'y': [95],
    'text': ['Russian Invasion ↓']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='red'
).encode(
    x='x:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 100])),
    text='text:N'
)

final_chart = chart + invasion_line + invasion_text

final_chart

alt.LayerChart(...)

In [ ]:
# Individual Comparison Charts

for term in top_terms:
    
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED events (left axis)
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # google trends (right axis)
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Ukraine: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"ukraine_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")


✓ Saved: ukraine_kyiv_attack_comparison.html
✓ Saved: ukraine_ukraine_war_comparison.html
✓ Saved: ukraine_zelensky_comparison.html


In [ ]:
# Key findings Summary

# Identify the invasion spike
invasion_month = merged[merged['month'] == '2022-02-01']
if not invasion_month.empty:
    inv_events = invasion_month['EVENTS'].values[0]
    inv_fatalities = invasion_month['FATALITIES'].values[0]
    print(f"\n FEBRUARY 2022 (Russian Invasion):")
    print(f"   ACLED Events: {inv_events:,}")
    print(f"   ACLED Fatalities: {inv_fatalities:,}")
    print(f"\n   Google Trends (Feb 2022):")
    for term in trends_data.keys():
        if term in invasion_month.columns:
            val = invasion_month[term].values[0]
            if pd.notna(val):
                print(f"   - {term:20s}: {val:.0f}/100")

# Recent spikes
print(f"\n AUGUST 2025 (Recent Spike):")
recent = merged[merged['month'] == '2025-08-01']
if not recent.empty:
    print(f"   ACLED Events: {recent['EVENTS'].values[0]:,}")
    print(f"   ACLED Fatalities: {recent['FATALITIES'].values[0]:,}")
    print(f"\n   Google Trends (Aug 2025):")
    for term in trends_data.keys():
        if term in recent.columns:
            val = recent[term].values[0]
            if pd.notna(val):
                print(f"   - {term:20s}: {val:.0f}/100")


 FEBRUARY 2022 (Russian Invasion):
   ACLED Events: 1,875
   ACLED Fatalities: 1,292

   Google Trends (Feb 2022):
   - Ukraine war         : 83/100
   - Ukraine news        : 100/100
   - Russia Ukraine      : 100/100
   - Russian invasion    : 100/100
   - Ukraine invasion    : 100/100
   - Ukraine conflict    : 100/100
   - Kyiv attack         : 99/100
   - Zelensky            : 67/100

 AUGUST 2025 (Recent Spike):
   ACLED Events: 7,577
   ACLED Fatalities: 6,158

   Google Trends (Aug 2025):
   - Ukraine war         : 22/100
   - Ukraine news        : 12/100
   - Russia Ukraine      : 6/100
   - Russian invasion    : 3/100
   - Ukraine invasion    : 4/100
   - Ukraine conflict    : 8/100
   - Kyiv attack         : 100/100
   - Zelensky            : 28/100


INTERPRETATION

1. INVASION SPIKE (Feb 2022):
   - Massive spike in both ACLED events AND search interest
   - Near-perfect correlation: public attention matched reality
   - Search terms peaked at 100 (maximum Google Trends value)

2. SUSTAINED CONFLICT (2022-2023):
   - ACLED events remain elevated throughout
   - Search interest gradually declines (fatigue/normalization)
   - Suggests "conflict fatigue" in public attention

3. RECENT ESCALATION (2025):
   - ACLED events surge again (Aug 2025: 7,577 events)
   - Search interest increases but doesn't match Feb 2022 levels
   - Public attention is more muted despite similar event intensity

4. CORRELATION PATTERNS:
   - Best correlations with "Ukraine war" and "Ukraine news" (broad terms)
   - Specific event terms (e.g., "Kyiv attack") show more reactive spikes
   - Most searches are REACTIVE (lag behind events) not predictive

5. ATTENTION DECAY:
   - Initial shock (Feb 2022) generated maximum attention
   - Ongoing violence receives less attention despite similar intensity
   - "Normalization" of conflict in public consciousness


## India

In [ ]:
# Filter for India, 2020-2025
india_acled = acled[
    (acled['COUNTRY'] == 'India') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level to match Google Trends
india_acled['month'] = india_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
india_monthly = india_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"\nACLED Data: {len(india_monthly)} months")
print(f"Date range: {india_monthly['month'].min()} to {india_monthly['month'].max()}")
print(f"Total events: {india_monthly['EVENTS'].sum():,}")
print(f"Total fatalities: {india_monthly['FATALITIES'].sum():,}")

india_monthly



ACLED Data: 70 months
Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
Total events: 125,521
Total fatalities: 6,660


month  EVENTS  FATALITIES
0  2020-01-01    1798          59
1  2020-02-01    1797         124
2  2020-03-01     742          98
3  2020-04-01     652         112
4  2020-05-01    1289         150
..        ...     ...         ...
65 2025-06-01    2036          87
66 2025-07-01    2406          82
67 2025-08-01    2551          61
68 2025-09-01    1892          79
69 2025-10-01    1173          28

[70 rows x 3 columns]

In [ ]:
# India trends files
trends_files = {
    'Delhi protest': 'India/Delhi_protests.csv',
    'farmers protest India': 'India/farmers_protest_India.csv',
    'India farmer protest': 'India/India_farmer_protest.csv',
    'India News': 'India/India_news.csv',
    'India Protest': 'India/India_protests.csv',
    'Modi Protests': 'India/Modi_protests.csv',
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)  # Skip the "Category" row
        df.columns = ['month', 'value']
        
        # convert month to datetime
        df['month'] = pd.to_datetime(df['month'])
        
        # handle '<1' values (convert to 0.5)
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'])
        
        trends_data[name] = df
        print(f"✓ Loaded: {name:20s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"✗ Error loading {name}: {e}")

✓ Loaded: Delhi protest        - 70 months, max=100
✓ Loaded: farmers protest India - 70 months, max=100.0
✓ Loaded: India farmer protest - 70 months, max=100
✓ Loaded: India News           - 70 months, max=100
✓ Loaded: India Protest        - 70 months, max=100
✓ Loaded: Modi Protests        - 70 months, max=100


In [ ]:
# Merge with ACLED data
merged = india_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"\n✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")


✓ Merged dataset: 70 months with 6 search terms


In [ ]:
merged

month  EVENTS  FATALITIES  Delhi protest  farmers protest India  \
0  2020-01-01    1798          59             26                    0.0   
1  2020-02-01    1797         124             86                    0.0   
2  2020-03-01     742          98             20                    0.0   
3  2020-04-01     652         112              3                    0.0   
4  2020-05-01    1289         150              3                    0.5   
..        ...     ...         ...            ...                    ...   
65 2025-06-01    2036          87              4                    1.0   
66 2025-07-01    2406          82             16                    1.0   
67 2025-08-01    2551          61             54                    2.0   
68 2025-09-01    1892          79            100                    3.0   
69 2025-10-01    1173          28             39                    2.0   

    India farmer protest  India News  India Protest  Modi Protests  
0                      0          14             10             57  
1                      0          14              5             23  
2                      0          39              3             13  
3                      0          30              2              0  
4                      0          23              3              0  
..                   ...         ...            ...            ...  
65                     1          10              6             12  
66                     0          13              8             10  
67                     2          37             31             42  
68                     2         100            100            100  
69                     1          69             35             22  

[70 rows x 9 columns]

In [ ]:
# Correlation

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        # Remove NaN values for correlation
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:  # Need sufficient data
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))


          Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
        Modi Protests        0.081944           -0.214657           70
        Delhi protest        0.062817           -0.150495           70
 India farmer protest        0.043018           -0.208679           70
        India Protest        0.038916           -0.173550           70
farmers protest India        0.006580           -0.213239           70
           India News       -0.185464           -0.035419           70


In [ ]:
# Time-Lagged Correlation Anaysis

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            # Positive lag: shift events forward (searches lag behind events)
            corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
        else:
            # Negative lag: shift searches forward (searches lead events)
            corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")



Modi Protests:
  Lag -3 months (searches LEAD  ): correlation = +0.086
  Lag -2 months (searches LEAD  ): correlation = +0.132
  Lag -1 months (searches LEAD  ): correlation = +0.103
  Lag +0 months (CONCURRENT     ): correlation = +0.082
  Lag +1 months (searches LAG   ): correlation = +0.103
  Lag +2 months (searches LAG   ): correlation = +0.132
  Lag +3 months (searches LAG   ): correlation = +0.086

 Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Delhi protest:
  Lag -3 months (searches LEAD  ): correlation = +0.181
  Lag -2 months (searches LEAD  ): correlation = +0.181
  Lag -1 months (searches LEAD  ): correlation = +0.098
  Lag +0 months (CONCURRENT     ): correlation = +0.063
  Lag +1 months (searches LAG   ): correlation = +0.098
  Lag +2 months (searches LAG   ): correlation = +0.181
  Lag +3 months (searches LAG   ): correlation = +0.181

 Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

India farmer protest:
  Lag -3 

In [ ]:
 
print("KEY SPIKE PERIODS")
 
# Top 5 event spikes
print("\nTop 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")


KEY SPIKE PERIODS

Top 5 Event Spikes:

August 2024:
  ACLED Events: 3,421
  ACLED Fatalities: 87
  Search Interest:
    - Delhi protest            : 8/100
    - farmers protest India    : 1/100
    - India farmer protest     : 2/100
    - India News               : 8/100
    - India Protest            : 5/100
    - Modi Protests            : 12/100

June 2024:
  ACLED Events: 2,839
  ACLED Fatalities: 141
  Search Interest:
    - Delhi protest            : 2/100
    - farmers protest India    : 2/100
    - India farmer protest     : 3/100
    - India News               : 11/100
    - India Protest            : 1/100
    - Modi Protests            : 12/100

July 2023:
  ACLED Events: 2,657
  ACLED Fatalities: 196
  Search Interest:
    - Delhi protest            : 2/100
    - farmers protest India    : 2/100
    - India farmer protest     : 1/100
    - India News               : 8/100
    - India Protest            : 2/100
    - Modi Protests            : 8/100

March 2025:
  ACLED Eve

## VISUALIZATIONS

In [ ]:
#  Visaulizations

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'India: Protest Events vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining farmers protests and political demonstration patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Add annotation for major farmers' protest (Nov 2020)
farmers_protest_date = pd.to_datetime('2020-11-01')
annotation_line = alt.Chart(pd.DataFrame({'date': [farmers_protest_date]})).mark_rule(
    color='green',
    strokeWidth=2,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [farmers_protest_date],
    'y': [103],
    'label': ['↓ Farmers Protests Begin (Nov 2020)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='green'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('india_acled_vs_trends.html')
print("✓ Saved: india_acled_vs_trends.html")


✓ Saved: india_acled_vs_trends.html


In [ ]:
main_chart

alt.LayerChart(...)

In [ ]:
# Individual Comparison Charts
 

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='green', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='green'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'India: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"india_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: india_modi_protests_comparison.html
✓ Saved: india_delhi_protest_comparison.html
✓ Saved: india_india_farmer_protest_comparison.html


In [ ]:
# Event Type Analysis

print("EVENT TYPE BREAKDOWN")

# Get event types for the entire period
event_type_summary = india_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Protests                      : 98,933 ( 78.8%)
  Riots                         : 12,529 ( 10.0%)
  Strategic developments        :  5,906 (  4.7%)
  Violence against civilians    :  3,955 (  3.2%)
  Battles                       :  3,369 (  2.7%)
  Explosions/Remote violence    :    829 (  0.7%)


In [ ]:
# Stat Summary

print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")

# Correlation matrix
print("\nPearson Correlation Matrix:")
corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
corr_matrix = merged[corr_cols].corr()
print(corr_matrix.to_string())

# Compare pre-protest and post-protest periods
pre_protest = merged[merged['month'] < '2020-11-01']
post_protest = merged[merged['month'] >= '2020-11-01']

print("COMPARISON: Pre vs Post Farmers Protests (Nov 2020)")
print(f"\nPre-Farmers Protests (Jan-Oct 2020):")
print(f"  Avg Events/Month: {pre_protest['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {pre_protest['FATALITIES'].mean():.1f}")

print(f"\nPost-Farmers Protests (Nov 2020-Oct 2025):")
print(f"  Avg Events/Month: {post_protest['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {post_protest['FATALITIES'].mean():.1f}")

change_events = ((post_protest['EVENTS'].mean() / pre_protest['EVENTS'].mean()) - 1) * 100
change_fatalities = ((post_protest['FATALITIES'].mean() / pre_protest['FATALITIES'].mean()) - 1) * 100
print(f"\nChange: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 1793
  Avg Fatalities/Month: 95.1
  Max Events (single month): 3,421 (August 2024)
  Max Fatalities (single month): 196 (July 2023)
  Avg Fatality Rate: 0.057 per event

Pearson Correlation Matrix:
                        EVENTS  FATALITIES  Modi Protests  Delhi protest  India farmer protest
EVENTS                1.000000    0.235027       0.081944       0.062817              0.043018
FATALITIES            0.235027    1.000000      -0.214657      -0.150495             -0.208679
Modi Protests         0.081944   -0.214657       1.000000       0.742934              0.271901
Delhi protest         0.062817   -0.150495       0.742934       1.000000              0.134033
India farmer protest  0.043018   -0.208679       0.271901       0.134033              1.000000
COMPARISON: Pre vs Post Farmers Protests (Nov 2020)

Pre-Farmers Protests (Jan-Oct 2020):
  Avg Events/Month: 1482
  Avg Fatalities/Month: 1

Key Findings Summary


1. EVENT CHARACTERISTICS:
   - India's conflict events are predominantly PROTESTS (not violent clashes)
   - Very low fatality rate compared to other countries
   - Sustained high event counts throughout 2020-2025

2. FARMERS PROTESTS (Nov 2020 - Feb 2021):
   - Major spike in both events and search interest
   - Clear correlation between protest activity and public attention
   - Search terms specifically mentioning "farmers protest" show strong peaks

3. CORRELATION PATTERNS:
   - Moderate correlations for protest-specific terms
   - Weaker correlations for general "India" terms
   - Searches appear more responsive to specific protest movements

4. ATTENTION PATTERNS:
   - Unlike Ukraine, India shows periodic spikes rather than sustained decline
   - Each new protest wave generates renewed attention
   - Suggests public interest is event-driven, not conflict-driven

5. GEOGRAPHIC SPECIFICITY:
   - "Delhi protest" terms may capture capital-focused movements
   - Regional protests may not generate same search interest


## Myanmar

In [ ]:
# Filter for Myanmar, 2020-2025
myanmar_acled = acled[
    (acled['COUNTRY'] == 'Myanmar') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
myanmar_acled['month'] = myanmar_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
myanmar_monthly = myanmar_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(myanmar_monthly)} months")
print(f"  Date range: {myanmar_monthly['month'].min()} to {myanmar_monthly['month'].max()}")
print(f"  Total events: {myanmar_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {myanmar_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = myanmar_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")

✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 86,872
  Total fatalities: 89,665

✓ Event type data loaded for detailed analysis


In [ ]:
# Myanmar trends files
trends_files = {
    'Myanmar coup': 'Myanmar CSVs/myanmar_coup.csv',
    'Myanmar military coup': 'Myanmar CSVs/myanmar_military_coup.csv',
    'Myanmar protests': 'Myanmar CSVs/myanmar_protests.csv',
    'Aung San Suu Kyi': 'Myanmar CSVs/aung_san_suu_kyi.csv',
    'Myanmar news': 'Myanmar CSVs/myanmar_news.csv',
    'Burma coup': 'Myanmar CSVs/burma_coup.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  Error loading {name}: {e}")



  Loaded: Myanmar coup              - 70 months, max=100.0
  Loaded: Myanmar military coup     - 70 months, max=100.0
  Loaded: Myanmar protests          - 70 months, max=100
  Loaded: Aung San Suu Kyi          - 70 months, max=100
  Loaded: Myanmar news              - 70 months, max=100
  Loaded: Burma coup                - 70 months, max=100


In [ ]:
# Merge with ACLED data
merged = myanmar_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
display(merged[sample_cols].head(10).to_string(index=False))


✓ Merged dataset: 70 months with 6 search terms

Sample of merged data:


'     month  EVENTS  FATALITIES  Myanmar coup  Myanmar military coup  Myanmar protests  Aung San Suu Kyi  Myanmar news  Burma coup\n2020-01-01     181          84           0.0                    0.0                 0                 6             7           1\n2020-02-01     270          92           0.5                    0.0                 0                 5             7           0\n2020-03-01     179         143           0.0                    0.0                 0                 4            10           0\n2020-04-01      97          57           0.5                    0.5                 0                 5            10           0\n2020-05-01     114          40           0.5                    0.5                 0                 4             7           1\n2020-06-01     107          31           0.5                    1.0                 0                 4             6           0\n2020-07-01      76          30           0.5                    1.0               

In [ ]:
# Correlations

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))



          Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Myanmar military coup        0.319070           -0.150468           70
         Myanmar coup        0.269710           -0.178932           70
           Burma coup        0.261533           -0.159840           70
     Myanmar protests        0.229560           -0.164718           70
     Aung San Suu Kyi        0.197996           -0.207020           70
         Myanmar news        0.162846           -0.228356           70


In [ ]:
# Time-Lagged Correlations Analysis

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")



Myanmar military coup:
  Lag -3 months (searches LEAD  ): correlation = +0.307
  Lag -2 months (searches LEAD  ): correlation = +0.310
  Lag -1 months (searches LEAD  ): correlation = +0.315
  Lag +0 months (CONCURRENT     ): correlation = +0.319
  Lag +1 months (searches LAG   ): correlation = +0.315
  Lag +2 months (searches LAG   ): correlation = +0.310
  Lag +3 months (searches LAG   ): correlation = +0.307

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Myanmar coup:
  Lag -3 months (searches LEAD  ): correlation = +0.264
  Lag -2 months (searches LEAD  ): correlation = +0.266
  Lag -1 months (searches LEAD  ): correlation = +0.268
  Lag +0 months (CONCURRENT     ): correlation = +0.270
  Lag +1 months (searches LAG   ): correlation = +0.268
  Lag +2 months (searches LAG   ): correlation = +0.266
  Lag +3 months (searches LAG   ): correlation = +0.264

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Burma coup:
  Lag 

In [ ]:
#  Indentify Key Periods

# The Coup: February 2021
print("\n📍 FEBRUARY 2021 (Military Coup):")
feb_2021 = merged[merged['month'] == '2021-02-01']
if not feb_2021.empty:
    print(f"  ACLED Events: {feb_2021['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {feb_2021['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Feb 2021):")
    for term in trends_data.keys():
        if term in feb_2021.columns:
            val = feb_2021[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Peak violence: May 2021
print("\n📍 MAY 2021 (Peak Violence):")
may_2021 = merged[merged['month'] == '2021-05-01']
if not may_2021.empty:
    print(f"  ACLED Events: {may_2021['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {may_2021['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (May 2021):")
    for term in trends_data.keys():
        if term in may_2021.columns:
            val = may_2021[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Recent escalation: December 2023
print("\n📍 DECEMBER 2023 (Recent Escalation):")
dec_2023 = merged[merged['month'] == '2023-12-01']
if not dec_2023.empty:
    print(f"  ACLED Events: {dec_2023['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {dec_2023['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Dec 2023):")
    for term in trends_data.keys():
        if term in dec_2023.columns:
            val = dec_2023[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")



📍 FEBRUARY 2021 (Military Coup):
  ACLED Events: 2,147
  ACLED Fatalities: 99

  Google Trends (Feb 2021):
    - Myanmar coup             : 100/100
    - Myanmar military coup    : 100/100
    - Myanmar protests         : 62/100
    - Aung San Suu Kyi         : 100/100
    - Myanmar news             : 100/100
    - Burma coup               : 100/100

📍 MAY 2021 (Peak Violence):
  ACLED Events: 2,455
  ACLED Fatalities: 971

  Google Trends (May 2021):
    - Myanmar coup             : 9/100
    - Myanmar military coup    : 14/100
    - Myanmar protests         : 16/100
    - Aung San Suu Kyi         : 8/100
    - Myanmar news             : 26/100
    - Burma coup               : 6/100

📍 DECEMBER 2023 (Recent Escalation):
  ACLED Events: 2,042
  ACLED Fatalities: 1,723

  Google Trends (Dec 2023):
    - Myanmar coup             : 2/100
    - Myanmar military coup    : 3/100
    - Myanmar protests         : 1/100
    - Aung San Suu Kyi         : 3/100
    - Myanmar news             : 8/

## VISUALIZATIONS

In [ ]:
# Visualzation

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Myanmar: Coup Impact and Civil Resistance (2020-2025)',
        'subtitle': 'Examining military coup, protest response, and armed resistance patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Annotation for military coup (Feb 1, 2021)
coup_date = pd.to_datetime('2021-02-01')
annotation_line = alt.Chart(pd.DataFrame({'date': [coup_date]})).mark_rule(
    color='darkred',
    strokeWidth=3,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [coup_date],
    'y': [103],
    'label': ['↓ Military Coup (Feb 1, 2021)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='darkred'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('myanmar_acled_vs_trends.html')
print("✓ Saved: myanmar_acled_vs_trends.html")


✓ Saved: myanmar_acled_vs_trends.html


In [ ]:
main_chart

alt.LayerChart(...)

In [ ]:
# Indivdual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='darkred', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='darkred'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Myanmar: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"myanmar_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: myanmar_myanmar_military_coup_comparison.html
✓ Saved: myanmar_myanmar_coup_comparison.html
✓ Saved: myanmar_burma_coup_comparison.html


In [ ]:
# Event Type Analysis 

print("EVENT TYPE BREAKDOWN")

# Event types for the entire period
event_type_summary = myanmar_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

# Event types over time
print("Event Evolution: Pre-Coup vs Post-Coup")

pre_coup = myanmar_acled[myanmar_acled['month'] < '2021-02-01']
post_coup = myanmar_acled[myanmar_acled['month'] >= '2021-02-01']

print("\nPre-Coup (2020-Jan 2021):")
pre_types = pre_coup.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in pre_types.items():
    pct = (count / pre_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

print("\nPost-Coup (Feb 2021-2025):")
post_types = post_coup.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in post_types.items():
    pct = (count / post_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")


EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Explosions/Remote violence    : 22,975 ( 26.4%)
  Battles                       : 20,943 ( 24.1%)
  Strategic developments        : 17,329 ( 19.9%)
  Protests                      : 15,386 ( 17.7%)
  Violence against civilians    : 10,154 ( 11.7%)
  Riots                         :     85 (  0.1%)
Event Evolution: Pre-Coup vs Post-Coup

Pre-Coup (2020-Jan 2021):
  Battles                       :    521 ( 29.8%)
  Explosions/Remote violence    :    356 ( 20.4%)
  Violence against civilians    :    323 ( 18.5%)
  Protests                      :    318 ( 18.2%)
  Strategic developments        :    206 ( 11.8%)
  Riots                         :     23 (  1.3%)

Post-Coup (Feb 2021-2025):
  Explosions/Remote violence    : 22,619 ( 26.6%)
  Battles                       : 20,422 ( 24.0%)
  Strategic developments        : 17,123 ( 20.1%)
  Protests                      : 15,068 ( 17.7%)
  Violence against civilians    :  9,831 ( 11.5%)


In [ ]:
# Stats Summary

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")

# Compare pre-coup and post-coup periods
pre_coup_merged = merged[merged['month'] < '2021-02-01']
post_coup_merged = merged[merged['month'] >= '2021-02-01']

print("COMPARISON: Pre vs Post Military Coup (Feb 1, 2021)")
print(f"\nPre-Coup (2020-Jan 2021):")
print(f"  Avg Events/Month: {pre_coup_merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {pre_coup_merged['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {pre_coup_merged['fatality_rate'].mean():.3f}")

print(f"\nPost-Coup (Feb 2021-2025):")
print(f"  Avg Events/Month: {post_coup_merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {post_coup_merged['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {post_coup_merged['fatality_rate'].mean():.3f}")

if len(pre_coup_merged) > 0 and len(post_coup_merged) > 0:
    change_events = ((post_coup_merged['EVENTS'].mean() / pre_coup_merged['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((post_coup_merged['FATALITIES'].mean() / pre_coup_merged['FATALITIES'].mean()) - 1) * 100
    print(f"\nChange: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())


Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 1241
  Avg Fatalities/Month: 1280.9
  Max Events (single month): 2,455 (May 2021)
  Max Fatalities (single month): 2,637 (April 2022)
  Avg Fatality Rate: 0.935 per event
COMPARISON: Pre vs Post Military Coup (Feb 1, 2021)

Pre-Coup (2020-Jan 2021):
  Avg Events/Month: 134
  Avg Fatalities/Month: 55.8
  Avg Fatality Rate: 0.403

Post-Coup (Feb 2021-2025):
  Avg Events/Month: 1493
  Avg Fatalities/Month: 1560.3
  Avg Fatality Rate: 1.056

Change: Events +1011.3%, Fatalities +2694.0%

Pearson Correlation Matrix:
                         EVENTS  FATALITIES  Myanmar military coup  Myanmar coup  Burma coup
EVENTS                 1.000000    0.781807               0.319070      0.269710    0.261533
FATALITIES             0.781807    1.000000              -0.150468     -0.178932   -0.159840
Myanmar military coup  0.319070   -0.150468               1.000000      0.990624    0.972329
Myanmar coup           0.269710   -0.178

In [ ]:
# Attention Decay Analysis

# Attention per event for key periods
feb_2021_data = merged[merged['month'] == '2021-02-01']
may_2021_data = merged[merged['month'] == '2021-05-01']
recent_data = merged[merged['month'] == '2023-12-01']

if len(top_terms) > 0 and not feb_2021_data.empty:
    best_term = top_terms[0]
    
    print(f"\nAttention Efficiency (using '{best_term}'):")
     
    
    if not feb_2021_data.empty and best_term in feb_2021_data.columns:
        feb_events = feb_2021_data['EVENTS'].values[0]
        feb_search = feb_2021_data[best_term].values[0]
        feb_efficiency = (feb_search / feb_events) * 100 if feb_events > 0 else 0
        print(f"Feb 2021 (Coup): {feb_events:,} events → {feb_search:.0f}/100 search = {feb_efficiency:.3f} per 100 events")
    
    if not may_2021_data.empty and best_term in may_2021_data.columns:
        may_events = may_2021_data['EVENTS'].values[0]
        may_search = may_2021_data[best_term].values[0]
        may_efficiency = (may_search / may_events) * 100 if may_events > 0 else 0
        print(f"May 2021 (Peak):  {may_events:,} events → {may_search:.0f}/100 search = {may_efficiency:.3f} per 100 events")
    
    if not recent_data.empty and best_term in recent_data.columns:
        rec_events = recent_data['EVENTS'].values[0]
        rec_search = recent_data[best_term].values[0]
        rec_efficiency = (rec_search / rec_events) * 100 if rec_events > 0 else 0
        print(f"Dec 2023 (Recent): {rec_events:,} events → {rec_search:.0f}/100 search = {rec_efficiency:.3f} per 100 events")



Attention Efficiency (using 'Myanmar military coup'):
Feb 2021 (Coup): 2,147 events → 100/100 search = 4.658 per 100 events
May 2021 (Peak):  2,455 events → 14/100 search = 0.570 per 100 events
Dec 2023 (Recent): 2,042 events → 3/100 search = 0.147 per 100 events


### Key Findings Summary

1. THE COUP MOMENT (Feb 2021):
   - Discrete political event generated massive immediate attention
   - Search interest peaked at coup announcement
   - Different from gradual escalation patterns (Ukraine, India)

2. PROTEST-TO-RESISTANCE TRANSITION:
   - Initial peaceful protests (Feb-Mar 2021)
   - Violent crackdown led to armed resistance
   - Event composition shifted from protests to battles

3. ATTENTION PATTERN:
   - Sharp spike at coup (Feb 2021)
   - Rapid decay despite escalating violence
   - Similar to Ukraine: discrete dramatic event > sustained conflict

4. FATALITY ESCALATION:
   - Low fatalities at coup (Feb 2021)
   - Peaked during crackdown (Jan 2022: 2,287 deaths)
   - Recent periods show sustained violence with minimal attention

5. SEARCH TERM PERFORMANCE:
   - Coup-specific terms peaked early
   - "Aung San Suu Kyi" tracked political narrative
   - General terms showed weaker correlation

## USA

In [20]:
# Filter for United States, 2020-2025
usa_acled = acled[
    (acled['COUNTRY'] == 'United States') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
usa_acled['month'] = usa_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
usa_monthly = usa_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(usa_monthly)} months")
print(f"  Date range: {usa_monthly['month'].min()} to {usa_monthly['month'].max()}")
print(f"  Total events: {usa_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {usa_monthly['FATALITIES'].sum():,}")

# Get event type breakdown
event_types = usa_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")



✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 91,922
  Total fatalities: 496

✓ Event type data loaded for detailed analysis


In [98]:
# USA trends files
trends_files = {
    'George Floyd': 'USA CSVs/george_floyd.csv',
    'Black Lives Matter': 'USA CSVs/black_lives_matter.csv',
    'BLM protests': 'USA CSVs/blm.csv',
    'Minneapolis protest': 'USA CSVs/minneapolis_protest.csv',
    'police brutality': 'USA CSVs/police_brutality.csv',
    'protests today': 'USA CSVs/protests_today.csv',
    'Derek Chauvin': 'USA CSVs/Derek_Chauvin.csv'

}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")


  ✓ Loaded: George Floyd              - 70 months, max=100.0
  ✓ Loaded: Black Lives Matter        - 70 months, max=100.0
  ✓ Loaded: BLM protests              - 70 months, max=100
  ✓ Loaded: Minneapolis protest       - 70 months, max=100.0
  ✓ Loaded: police brutality          - 70 months, max=100
  ✓ Loaded: protests today            - 70 months, max=100
  ✓ Loaded: Derek Chauvin             - 70 months, max=100.0


In [147]:
# Merge with ACLED data
merged = usa_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))


✓ Merged dataset: 70 months with 5 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Syria civil war  Syria news  Aleppo  Syrian conflict  Syria attack
2020-01-01    1149           3               15          19      33               39            33
2020-02-01     838           5               33          41      38               51            31
2020-03-01     328           9               19          36      27               46            30
2020-04-01     675           7               10          14      31               30            20
2020-05-01    4454          19               10          12      32               32            16
2020-06-01    4736          15                8          12      33               28            16
2020-07-01    2170           7                7          10      31               20            12
2020-08-01    2764          12                6          11      30               21            16
2020-09-01    2033           2       

In [148]:
# Correlations
correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))



    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Syrian conflict       -0.005977            0.157371           70
   Syria attack       -0.011564           -0.029646           70
     Syria news       -0.020008            0.155877           70
         Aleppo       -0.052144            0.031217           70
Syria civil war       -0.122412            0.180526           70


In [149]:
# Time-Lagged Correlation Analysis

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")



Syrian conflict:
  Lag -3 months (searches LEAD  ): correlation = +0.038
  Lag -2 months (searches LEAD  ): correlation = +0.005
  Lag -1 months (searches LEAD  ): correlation = -0.023
  Lag +0 months (CONCURRENT     ): correlation = -0.006
  Lag +1 months (searches LAG   ): correlation = -0.023
  Lag +2 months (searches LAG   ): correlation = +0.005
  Lag +3 months (searches LAG   ): correlation = +0.038

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Syria attack:
  Lag -3 months (searches LEAD  ): correlation = +0.002
  Lag -2 months (searches LEAD  ): correlation = -0.004
  Lag -1 months (searches LEAD  ): correlation = -0.016
  Lag +0 months (CONCURRENT     ): correlation = -0.012
  Lag +1 months (searches LAG   ): correlation = -0.016
  Lag +2 months (searches LAG   ): correlation = -0.004
  Lag +3 months (searches LAG   ): correlation = +0.002

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Syria news:
  Lag -3 mon

In [150]:
#  Indentify Key Periods

# George Floyd protests: May-June 2020
print("\n MAY 2020 (George Floyd Death):")
may_2020 = merged[merged['month'] == '2020-05-01']
if not may_2020.empty:
    print(f"  ACLED Events: {may_2020['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {may_2020['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (May 2020):")
    for term in trends_data.keys():
        if term in may_2020.columns:
            val = may_2020[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n JUNE 2020 (Peak Protests):")
june_2020 = merged[merged['month'] == '2020-06-01']
if not june_2020.empty:
    print(f"  ACLED Events: {june_2020['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {june_2020['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (June 2020):")
    for term in trends_data.keys():
        if term in june_2020.columns:
            val = june_2020[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n AUGUST 2020 (Sustained Movement):")
aug_2020 = merged[merged['month'] == '2020-08-01']
if not aug_2020.empty:
    print(f"  ACLED Events: {aug_2020['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {aug_2020['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Aug 2020):")
    for term in trends_data.keys():
        if term in aug_2020.columns:
            val = aug_2020[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Recent period: June 2025
print("\n JUNE 2025 (Recent Spike):")
june_2025 = merged[merged['month'] == '2025-06-01']
if not june_2025.empty:
    print(f"  ACLED Events: {june_2025['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {june_2025['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (June 2025):")
    for term in trends_data.keys():
        if term in june_2025.columns:
            val = june_2025[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")



 MAY 2020 (George Floyd Death):
  ACLED Events: 4,454
  ACLED Fatalities: 19

  Google Trends (May 2020):
    - Syria civil war          : 10/100
    - Syria news               : 12/100
    - Aleppo                   : 32/100
    - Syrian conflict          : 32/100
    - Syria attack             : 16/100

 JUNE 2020 (Peak Protests):
  ACLED Events: 4,736
  ACLED Fatalities: 15

  Google Trends (June 2020):
    - Syria civil war          : 8/100
    - Syria news               : 12/100
    - Aleppo                   : 33/100
    - Syrian conflict          : 28/100
    - Syria attack             : 16/100

 AUGUST 2020 (Sustained Movement):
  ACLED Events: 2,764
  ACLED Fatalities: 12

  Google Trends (Aug 2020):
    - Syria civil war          : 6/100
    - Syria news               : 11/100
    - Aleppo                   : 30/100
    - Syrian conflict          : 21/100
    - Syria attack             : 16/100

 JUNE 2025 (Recent Spike):
  ACLED Events: 2,766
  ACLED Fatalities: 3

  Google

## VISUALIZATIONS

In [151]:
# Visualizations

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'USA: George Floyd Protests and Public Search Interest (2020-2025)',
        'subtitle': 'Examining racial justice protests and public attention patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Annotation for George Floyd death (May 25, 2020)
floyd_date = pd.to_datetime('2020-05-25')
annotation_line = alt.Chart(pd.DataFrame({'date': [floyd_date]})).mark_rule(
    color='black',
    strokeWidth=3,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [floyd_date],
    'y': [103],
    'label': ['↓ George Floyd Death (May 25, 2020)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='black'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('usa_acled_vs_trends.html')
print("✓ Saved: usa_acled_vs_trends.html")


✓ Saved: usa_acled_vs_trends.html


In [152]:
# Individual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='black', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='black'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'USA: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"usa_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")


✓ Saved: usa_syrian_conflict_comparison.html
✓ Saved: usa_syria_attack_comparison.html
✓ Saved: usa_syria_news_comparison.html


In [153]:
# Event Type Analysis

# Event types for the entire period
event_type_summary = usa_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

# Event types comparison: 2020 vs other years
print("Event Evolution: 2020 (Floyd Protests) vs 2021-2025")

year_2020 = usa_acled[usa_acled['month'].dt.year == 2020]
post_2020 = usa_acled[usa_acled['month'].dt.year > 2020]

print("\n2020 (George Floyd Protests):")
y2020_types = year_2020.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in y2020_types.items():
    pct = (count / y2020_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

print("\n2021-2025 (Post-Floyd):")
post_types = post_2020.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in post_types.items():
    pct = (count / post_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")


Total Events by Type (2020-2025):
  Protests                      : 85,540 ( 93.1%)
  Strategic developments        :  4,308 (  4.7%)
  Riots                         :  1,604 (  1.7%)
  Violence against civilians    :    420 (  0.5%)
  Battles                       :     31 (  0.0%)
  Explosions/Remote violence    :     19 (  0.0%)
Event Evolution: 2020 (Floyd Protests) vs 2021-2025

2020 (George Floyd Protests):
  Protests                      : 21,565 ( 92.8%)
  Riots                         :    832 (  3.6%)
  Strategic developments        :    703 (  3.0%)
  Violence against civilians    :    114 (  0.5%)
  Battles                       :     10 (  0.0%)
  Explosions/Remote violence    :      3 (  0.0%)

2021-2025 (Post-Floyd):
  Protests                      : 63,975 ( 93.1%)
  Strategic developments        :  3,605 (  5.2%)
  Riots                         :    772 (  1.1%)
  Violence against civilians    :    306 (  0.4%)
  Battles                       :     21 (  0.0%)
  Explo

In [182]:
# Statistical Summary

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.4f} per event")

# Compare 2020 Floyd protests to baseline
year_2020_merged = merged[merged['month'].dt.year == 2020]
other_years = merged[merged['month'].dt.year != 2020]

 
print("COMPARISON: 2020 (George Floyd) vs 2021-2025 (Baseline)")
print(f"\n2020 (Floyd Protest Year):")
print(f"  Avg Events/Month: {year_2020_merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {year_2020_merged['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {year_2020_merged['fatality_rate'].mean():.4f}")

print(f"\n2021-2025 (Other Years):")
print(f"  Avg Events/Month: {other_years['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {other_years['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {other_years['fatality_rate'].mean():.4f}")

if len(year_2020_merged) > 0 and len(other_years) > 0:
    change_events = ((year_2020_merged['EVENTS'].mean() / other_years['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((year_2020_merged['FATALITIES'].mean() / other_years['FATALITIES'].mean()) - 1) * 100
    print(f"\n2020 vs Others: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())


Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 978
  Avg Fatalities/Month: 1005.2
  Max Events (single month): 2,930 (October 2023)
  Max Fatalities (single month): 9,773 (October 2023)
  Avg Fatality Rate: 0.5300 per event
COMPARISON: 2020 (George Floyd) vs 2021-2025 (Baseline)

2020 (Floyd Protest Year):
  Avg Events/Month: 395
  Avg Fatalities/Month: 4.4
  Avg Fatality Rate: 0.0103

2021-2025 (Other Years):
  Avg Events/Month: 1098
  Avg Fatalities/Month: 1212.3
  Avg Fatality Rate: 0.6375

2020 vs Others: Events -64.0%, Fatalities -99.6%

Pearson Correlation Matrix:
                      EVENTS  FATALITIES  Gaza war      Gaza  Israel Gaza attack
EVENTS              1.000000    0.873639  0.765360  0.709144            0.599379
FATALITIES          0.873639    1.000000  0.824808  0.786207            0.721554
Gaza war            0.765360    0.824808  1.000000  0.965118            0.856999
Gaza                0.709144    0.786207  0.965118  1.000000            0.

Key Findings Summary

1. THE GEORGE FLOYD MOMENT (May-June 2020):
   - Specific incident (police killing) triggered nationwide protests
   - Search interest peaked simultaneously with events
   - Domestic USA event with near-perfect attention alignment

2. EVENT CHARACTERISTICS:
   - Predominantly protests (93% in 2020)
   - Extremely low fatality rate (0.004 per event)
   - Concentrated in 2-3 month period (May-Aug 2020)

3. CORRELATION PATTERNS:
   - Expected to show strong correlation (domestic event)
   - Person-specific: "George Floyd" as search anchor
   - Movement-specific: "Black Lives Matter" as ideology anchor

4. ATTENTION DECAY:
   - Rapid decline from June 2020 peak
   - Similar pattern to international events despite domestic location
   - Ongoing issues (police violence) generate periodic small spikes

5. COMPARISON TO OTHER CASES:
   - USA: Domestic crisis with direct victim
   - Myanmar: Distant coup with celebrity leader
   - Ukraine: International war with geopolitical stakes
   - India: Routine democratic protests

## Mexico

In [252]:
# MEXICO VIOLENCE ANALYSIS: ACLED EVENTS vs GOOGLE TRENDS

# Filter for Mexico, 2020-2025
mexico_acled = acled[
    (acled['COUNTRY'] == 'Mexico') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
mexico_acled['month'] = mexico_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
mexico_monthly = mexico_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(mexico_monthly)} months")
print(f"  Date range: {mexico_monthly['month'].min()} to {mexico_monthly['month'].max()}")
print(f"  Total events: {mexico_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {mexico_monthly['FATALITIES'].sum():,}")

# Event type breakdown
event_types = mexico_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")

✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 84,682
  Total fatalities: 46,812

✓ Event type data loaded for detailed analysis


In [253]:
# Mexico violence/cartel trends files
trends_files = {
    'Mexico violence': 'Mexico/mexico_violence.csv',
    'Mexico cartel': 'Mexico/mexico_cartel.csv',
    'cartel war Mexico': 'Mexico/cartel_war_mexico.csv',
    'drug cartel Mexico': 'Mexico/drug_cartel_mexico.csv',
    'Mexico news': 'Mexico/mexico_news.csv',
    'Mexico security': 'Mexico/mexico_security.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")



  ✓ Loaded: Mexico violence           - 70 months, max=100
  ✓ Loaded: Mexico cartel             - 70 months, max=100
  ✓ Loaded: cartel war Mexico         - 70 months, max=100
  ✓ Loaded: drug cartel Mexico        - 70 months, max=100
  ✓ Loaded: Mexico news               - 70 months, max=100
  ✓ Loaded: Mexico security           - 70 months, max=100


In [254]:
 # 3. Merge ACLED & Trends Data

# Merge with ACLED data
merged = mexico_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))


✓ Merged dataset: 70 months with 6 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Mexico violence  Mexico cartel  cartel war Mexico  drug cartel Mexico  Mexico news  Mexico security
2020-01-01     964         662               11             31                  7                  38            5                6
2020-02-01    1372         874               13             63                 12                  90            5                7
2020-03-01     941         598               11             33                  8                  55           13                6
2020-04-01     949         613               10             31                  8                  62            8                5
2020-05-01    1394         960                9             29                  6                  42            6                5
2020-06-01    1239         813                9             24                  6                  36            6                5
2020

In [255]:
# Correlations

print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

CORRELATION ANALYSIS: Search Terms vs ACLED Events

       Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
   Mexico security        0.081107           -0.140701           70
     Mexico cartel        0.048654           -0.119556           70
   Mexico violence        0.033571           -0.199125           70
       Mexico news       -0.015532           -0.197917           70
 cartel war Mexico       -0.037638           -0.172374           70
drug cartel Mexico       -0.103896           -0.091692           70


In [256]:
# Time-Lagged Correlation Analysis

print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)

Mexico security:
  Lag -3 months (searches LEAD  ): correlation = +0.313
  Lag -2 months (searches LEAD  ): correlation = +0.209
  Lag -1 months (searches LEAD  ): correlation = +0.179
  Lag +0 months (CONCURRENT     ): correlation = +0.081
  Lag +1 months (searches LAG   ): correlation = +0.179
  Lag +2 months (searches LAG   ): correlation = +0.209
  Lag +3 months (searches LAG   ): correlation = +0.313

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Mexico cartel:
  Lag -3 months (searches LEAD  ): correlation = +0.014
  Lag -2 months (searches LEAD  ): correlation = +0.053
  Lag -1 months (searches LEAD  ): correlation = +0.084
  Lag +0 months (CONCURRENT     ): correlation = +0.049
  Lag +1 months (searches LAG   ): correlation = +0.084
  Lag +2 months (searches LAG   ): correlation = +0

In [257]:

# Identitfy Key Periods

print("KEY PERIODS")

# Top 5 event spikes
print("\nTop 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# Top 5 fatality spikes
 
print("Top 5 Fatality Spikes:")
 
top_fatal = merged.nlargest(5, 'FATALITIES')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_fatal.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Fatality Rate: {(row['FATALITIES']/row['EVENTS']):.2f} per event")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

KEY PERIODS

Top 5 Event Spikes:

March 2025:
  ACLED Events: 1,903
  ACLED Fatalities: 852
  Search Interest:
    - Mexico violence          : 14/100
    - Mexico cartel            : 45/100
    - cartel war Mexico        : 13/100
    - drug cartel Mexico       : 39/100
    - Mexico news              : 5/100
    - Mexico security          : 10/100

May 2025:
  ACLED Events: 1,801
  ACLED Fatalities: 821
  Search Interest:
    - Mexico violence          : 12/100
    - Mexico cartel            : 32/100
    - cartel war Mexico        : 9/100
    - drug cartel Mexico       : 28/100
    - Mexico news              : 2/100
    - Mexico security          : 8/100

February 2025:
  ACLED Events: 1,643
  ACLED Fatalities: 701
  Search Interest:
    - Mexico violence          : 13/100
    - Mexico cartel            : 43/100
    - cartel war Mexico        : 15/100
    - drug cartel Mexico       : 38/100
    - Mexico news              : 6/100
    - Mexico security          : 12/100

November 2024:
 

## VISUALIZATIONS

In [258]:
# Visualizations

 

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Mexico: Cartel Violence vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining organized crime violence and normalized conflict patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('mexico_acled_vs_trends.html')
print("✓ Saved: mexico_acled_vs_trends.html")

✓ Saved: mexico_acled_vs_trends.html


In [259]:
# Indiviual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='darkgreen', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='darkgreen'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Mexico: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"mexico_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")


✓ Saved: mexico_mexico_security_comparison.html
✓ Saved: mexico_mexico_cartel_comparison.html
✓ Saved: mexico_mexico_violence_comparison.html


In [260]:

# Event Type Analysis

print("EVENT TYPE BREAKDOWN")

# Get event types for the entire period
event_type_summary = mexico_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Protests                      : 34,938 ( 41.3%)
  Violence against civilians    : 32,418 ( 38.3%)
  Battles                       :  7,665 (  9.1%)
  Strategic developments        :  5,530 (  6.5%)
  Riots                         :  3,880 (  4.6%)
  Explosions/Remote violence    :    251 (  0.3%)


In [261]:
# Statistical Summary

print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")
print(f"  Median Fatality Rate: {merged['fatality_rate'].median():.3f} per event")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

# Analyze temporal trends
 
print("TEMPORAL TRENDS (by Year)")
print("-"*80)

yearly_stats = merged.copy()
yearly_stats['year'] = yearly_stats['month'].dt.year
yearly_summary = yearly_stats.groupby('year').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
yearly_summary['fatality_rate'] = yearly_summary['FATALITIES'] / yearly_summary['EVENTS']

print("\nYearly Summary:")
print(yearly_summary.to_string(index=False))



STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 1210
  Avg Fatalities/Month: 668.7
  Max Events (single month): 1,903 (March 2025)
  Max Fatalities (single month): 960 (May 2020)
  Avg Fatality Rate: 0.557 per event
  Median Fatality Rate: 0.557 per event

Pearson Correlation Matrix:
                   EVENTS  FATALITIES  Mexico security  Mexico cartel  Mexico violence
EVENTS           1.000000    0.705235         0.081107       0.048654         0.033571
FATALITIES       0.705235    1.000000        -0.140701      -0.119556        -0.199125
Mexico security  0.081107   -0.140701         1.000000       0.356148         0.917101
Mexico cartel    0.048654   -0.119556         0.356148       1.000000         0.569907
Mexico violence  0.033571   -0.199125         0.917101       0.569907         1.000000
TEMPORAL TRENDS (by Year)
--------------------------------------------------------------------------------

Yearly Summary:
 year  EVENTS  FATALITIES

In [262]:
# Normalization Analysis

print("NORMALIZATION ANALYSIS: The 'Background Violence' Pattern")

# Calculate coefficient of variation for events and search
events_cv = merged['EVENTS'].std() / merged['EVENTS'].mean()
search_cv = merged[top_terms[0]].std() / merged[top_terms[0]].mean() if len(top_terms) > 0 else 0

print(f"\nCoefficient of Variation:")
print(f"  Events:  {events_cv:.3f} (variability in event counts)")
print(f"  Searches: {search_cv:.3f} (variability in search interest)")

if events_cv < 0.3:
    print("\n→ Low event variability suggests NORMALIZED violence")
    print("  (consistent high levels with no dramatic spikes)")
else:
    print("\n→ High event variability suggests EPISODIC violence")
    print("  (distinct periods of escalation and de-escalation)")



NORMALIZATION ANALYSIS: The 'Background Violence' Pattern

Coefficient of Variation:
  Events:  0.175 (variability in event counts)
  Searches: 1.826 (variability in search interest)

→ Low event variability suggests NORMALIZED violence
  (consistent high levels with no dramatic spikes)


### Mexico Analysis - Key Findings

**Correlation:** r = 0.08 (negligible)  
**Deaths (2020-2025):** 46,812  
**Pattern:** Normalized Violence - framed as "crime" not "conflict"  

**What This Means:**
Mexico's 46,812 deaths—exceeding Syria's 41,178—generate almost no attention (r = 0.08) due to **media framing effects**. Violence is portrayed as "cartel crime" rather than "armed conflict," rendering it invisible despite death tolls comparable to recognized wars.

The negligible correlation demonstrates how **categorical framing** determines attention: when violence is classified as "domestic crime" rather than "humanitarian crisis," it disappears from global attention hierarchies regardless of severity. This shows that attention is not just about death counts but about how violence is categorized and presented.

## Brazil

In [263]:

# Filter for Brazil, 2020-2025
brazil_acled = acled[
    (acled['COUNTRY'] == 'Brazil') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
brazil_acled['month'] = brazil_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
brazil_monthly = brazil_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(brazil_monthly)} months")
print(f"  Date range: {brazil_monthly['month'].min()} to {brazil_monthly['month'].max()}")
print(f"  Total events: {brazil_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {brazil_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = brazil_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")


✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 72,011
  Total fatalities: 41,686

✓ Event type data loaded for detailed analysis


In [264]:

# Brazil election/protest trends files
trends_files = {
    'Brazil election': 'Brazil/brazil_election.csv',
    'Bolsonaro Lula': 'Brazil/bolsonaro_lula.csv',
    'Brazil protests': 'Brazil/brazil_protests.csv',
    'Lula Brazil': 'Brazil/lula_brazil.csv',
    'Bolsonaro': 'Brazil/bolsonaro.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")


  ✓ Loaded: Brazil election           - 70 months, max=100.0
  ✓ Loaded: Bolsonaro Lula            - 70 months, max=100.0
  ✓ Loaded: Brazil protests           - 70 months, max=100
  ✓ Loaded: Lula Brazil               - 70 months, max=100
  ✓ Loaded: Bolsonaro                 - 70 months, max=100


In [ ]:
# 3. Merge ACLED & Trends Data
 
# Merge with ACLED data
merged = brazil_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))

✓ Merged dataset: 70 months with 5 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Brazil election  Bolsonaro Lula  Brazil protests  Lula Brazil  Bolsonaro
2020-01-01     999         618              0.5             1.0                3            2          8
2020-02-01    1107         587              0.5             1.0                2            2          8
2020-03-01    1018         424              0.5             1.0                4            2         32
2020-04-01     926         481              0.5             1.0                6            2         32
2020-05-01     988         466              1.0             1.0                8            2         28
2020-06-01     840         380              1.0             1.0               24            2         17
2020-07-01     798         385              1.0             1.0                5            2         21
2020-08-01    1090         490              1.0             1.0                4        

In [ ]:
# Correlations
 
print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")
 

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

CORRELATION ANALYSIS: Search Terms vs ACLED Events

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
 Bolsonaro Lula        0.783146            0.193477           70
Brazil election        0.755044            0.147047           70
      Bolsonaro        0.714499           -0.022968           70
    Lula Brazil        0.607458            0.249959           70
Brazil protests        0.047326           -0.043514           70


In [ ]:
# Time-Lagged Correlation Analysis
 
print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
 
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")
print("-"*80)

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)
--------------------------------------------------------------------------------

Bolsonaro Lula:
  Lag -3 months (searches LEAD  ): correlation = +0.795
  Lag -2 months (searches LEAD  ): correlation = +0.793
  Lag -1 months (searches LEAD  ): correlation = +0.791
  Lag +0 months (CONCURRENT     ): correlation = +0.783
  Lag +1 months (searches LAG   ): correlation = +0.791
  Lag +2 months (searches LAG   ): correlation = +0.793
  Lag +3 months (searches LAG   ): correlation = +0.795

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Brazil election:
  Lag -3 months (searches LEAD  ): correlation = +0.774
  Lag -2 months (searches LEAD  ): correlation = +0.775
  Lag -1 months (searches LEAD  ): correlation = +0.772
  Lag +0 months (CONCURRENT     ): correlation = +0.755
  Lag +1 months (searches

In [ ]:
# 6.  Indentify Key Periods
 
print("KEY PERIODS")
 
# 2022 Election period: September-October
print("\n SEPTEMBER 2022 (Pre-Election Month):")
sep_2022 = merged[merged['month'] == '2022-09-01']
if not sep_2022.empty:
    print(f"  ACLED Events: {sep_2022['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {sep_2022['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Sep 2022):")
    for term in trends_data.keys():
        if term in sep_2022.columns:
            val = sep_2022[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n OCTOBER 2022 (Peak Election/Protest Month):")
oct_2022 = merged[merged['month'] == '2022-10-01']
if not oct_2022.empty:
    print(f"  ACLED Events: {oct_2022['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {oct_2022['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Oct 2022):")
    for term in trends_data.keys():
        if term in oct_2022.columns:
            val = oct_2022[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n JANUARY 2023 (Post-Election Protests - Brasília Riots):")
jan_2023 = merged[merged['month'] == '2023-01-01']
if not jan_2023.empty:
    print(f"  ACLED Events: {jan_2023['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {jan_2023['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Jan 2023):")
    for term in trends_data.keys():
        if term in jan_2023.columns:
            val = jan_2023[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")


KEY PERIODS

📍 SEPTEMBER 2022 (Pre-Election Month):
  ACLED Events: 1,222
  ACLED Fatalities: 640

  Google Trends (Sep 2022):
    - Brazil election          : 6/100
    - Bolsonaro Lula           : 21/100
    - Brazil protests          : 2/100
    - Lula Brazil              : 12/100
    - Bolsonaro                : 33/100

📍 OCTOBER 2022 (Peak Election/Protest Month):
  ACLED Events: 2,637
  ACLED Fatalities: 737

  Google Trends (Oct 2022):
    - Brazil election          : 100/100
    - Bolsonaro Lula           : 100/100
    - Brazil protests          : 5/100
    - Lula Brazil              : 100/100
    - Bolsonaro                : 100/100

📍 JANUARY 2023 (Post-Election Protests - Brasília Riots):
  ACLED Events: 1,023
  ACLED Fatalities: 670

  Google Trends (Jan 2023):
    - Brazil election          : 3/100
    - Bolsonaro Lula           : 9/100
    - Brazil protests          : 36/100
    - Lula Brazil              : 32/100
    - Bolsonaro                : 19/100
Top 5 Event Spikes

## VISUALIZATIONS

In [269]:
# Visualizations

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Brazil: Political Violence vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining electoral protests and political polarization patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Annotation for 2022 election (October)
election_date = pd.to_datetime('2022-10-01')
annotation_line = alt.Chart(pd.DataFrame({'date': [election_date]})).mark_rule(
    color='green',
    strokeWidth=3,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [election_date],
    'y': [103],
    'label': ['↓ Presidential Election (Oct 2022)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='green'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('brazil_acled_vs_trends.html')
print("✓ Saved: brazil_acled_vs_trends.html")

✓ Saved: brazil_acled_vs_trends.html


In [270]:
main_chart

alt.LayerChart(...)

In [271]:
# Individual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='green', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='green'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Brazil: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"brazil_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: brazil_bolsonaro_lula_comparison.html
✓ Saved: brazil_brazil_election_comparison.html
✓ Saved: brazil_bolsonaro_comparison.html


In [272]:
# EVENT TYPE ANALYSIS

 
print("EVENT TYPE BREAKDOWN")
 

# Get event types for the entire period
event_type_summary = brazil_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

# Event types comparison: 2022 (election) vs other years
 
print("Event Evolution: 2022 (Election Year) vs Others")
print("-"*80)

year_2022 = brazil_acled[brazil_acled['month'].dt.year == 2022]
other_years = brazil_acled[brazil_acled['month'].dt.year != 2022]

print("\n2022 (Election Year):")
y2022_types = year_2022.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in y2022_types.items():
    pct = (count / y2022_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

print("\nOther Years (2020, 2021, 2023-2025):")
other_types = other_years.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
for event_type, count in other_types.items():
    pct = (count / other_types.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Battles                       : 27,076 ( 37.6%)
  Violence against civilians    : 24,250 ( 33.7%)
  Protests                      : 14,188 ( 19.7%)
  Riots                         :  3,455 (  4.8%)
  Strategic developments        :  2,706 (  3.8%)
  Explosions/Remote violence    :    336 (  0.5%)
Event Evolution: 2022 (Election Year) vs Others
--------------------------------------------------------------------------------

2022 (Election Year):
  Violence against civilians    :  4,908 ( 33.2%)
  Battles                       :  4,816 ( 32.6%)
  Protests                      :  3,752 ( 25.4%)
  Riots                         :    885 (  6.0%)
  Strategic developments        :    375 (  2.5%)
  Explosions/Remote violence    :     45 (  0.3%)

Other Years (2020, 2021, 2023-2025):
  Battles                       : 22,260 ( 38.9%)
  Violence against civilians    : 19,342 ( 33.8%)
  Protests                      : 10,436 ( 18.2%)
  Ri

In [273]:


# STATISTICAL SUMMARY

print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")

# Compare 2022 election year to others
year_2022_merged = merged[merged['month'].dt.year == 2022]
other_years_merged = merged[merged['month'].dt.year != 2022]

 
print("COMPARISON: 2022 (Election Year) vs Others")
print("-"*80)
print(f"\n2022 (Election Year):")
print(f"  Avg Events/Month: {year_2022_merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {year_2022_merged['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {year_2022_merged['fatality_rate'].mean():.3f}")

print(f"\nOther Years:")
print(f"  Avg Events/Month: {other_years_merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {other_years_merged['FATALITIES'].mean():.1f}")
print(f"  Avg Fatality Rate: {other_years_merged['fatality_rate'].mean():.3f}")

if len(year_2022_merged) > 0 and len(other_years_merged) > 0:
    change_events = ((year_2022_merged['EVENTS'].mean() / other_years_merged['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((year_2022_merged['FATALITIES'].mean() / other_years_merged['FATALITIES'].mean()) - 1) * 100
    print(f"\n2022 vs Others: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 1029
  Avg Fatalities/Month: 595.5
  Max Events (single month): 2,637 (October 2022)
  Max Fatalities (single month): 877 (September 2023)
  Avg Fatality Rate: 0.590 per event
COMPARISON: 2022 (Election Year) vs Others
--------------------------------------------------------------------------------

2022 (Election Year):
  Avg Events/Month: 1232
  Avg Fatalities/Month: 656.1
  Avg Fatality Rate: 0.564

Other Years:
  Avg Events/Month: 987
  Avg Fatalities/Month: 583.0
  Avg Fatality Rate: 0.596

2022 vs Others: Events +24.8%, Fatalities +12.5%

Pearson Correlation Matrix:
                   EVENTS  FATALITIES  Bolsonaro Lula  Brazil election  Bolsonaro
EVENTS           1.000000    0.461067        0.783146         0.755044   0.714499
FATALITIES       0.461067    1.000000        0.193477         0.147047  -0.022968
Bolsonaro Lula   0.783146    0.193477        1.000000         0.967704   0.872084
B

### Key Findings Summary
 
1. ELECTORAL VIOLENCE PATTERN (2022):
   - Sharp spike in October 2022 (2,637 events)
   - Concentrated around presidential election
   - Mix of political protests and criminal violence

2. EVENT CHARACTERISTICS:
   - Violence against civilians dominates (~38%)
   - Protests concentrated in election periods
   - Ongoing criminal violence as baseline

3. CORRELATION PATTERNS:
   - Expected moderate correlation (r = 0.15-0.25)
   - Election-specific terms should peak in 2022
   - Political search (Bolsonaro/Lula) vs. violence search

4. POST-ELECTION PERIOD (Jan 2023):
   - Brasília riots (Jan 8, 2023) following election
   - Test if discrete political event generates attention
   - Similar to US Capitol riots pattern

5. COMPARISON TO OTHER CASES:
   - Brazil: Electoral + criminal violence mix
   - Mexico: Pure criminal violence (normalized)
   - India: Democratic protests (episodic)
   - USA: Single-event protest movement
   - Myanmar: Coup → civil war
   
6. THE "DUAL VIOLENCE" PATTERN:
   - Background criminal violence (like Mexico)
   - Episodic political violence (elections)
   - Tests if political framing increases attention

## Pakistan

In [274]:

acled['WEEK'] = pd.to_datetime(acled['WEEK'])

# Filter for Pakistan, 2020-2025
pakistan_acled = acled[
    (acled['COUNTRY'] == 'Pakistan') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
pakistan_acled['month'] = pakistan_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
pakistan_monthly = pakistan_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(pakistan_monthly)} months")
print(f"  Date range: {pakistan_monthly['month'].min()} to {pakistan_monthly['month'].max()}")
print(f"  Total events: {pakistan_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {pakistan_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = pakistan_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")

✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 53,662
  Total fatalities: 13,782

✓ Event type data loaded for detailed analysis


In [275]:

 # Load & Prepare Data

# Pakistan political crisis trends files
trends_files = {
    'Pakistan protests': 'Pakistan/pakistan_protests.csv',
    'Imran Khan': 'Pakistan/imran_khan.csv',
    'Pakistan news': 'Pakistan/pakistan_news.csv',
    'Pakistan election': 'Pakistan/pakistan_election.csv',
    'PTI Pakistan': 'Pakistan/pti_pakistan.csv',
    'Pakistan political crisis': 'Pakistan/pakistan_political_crisis.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:30s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")

  ✓ Loaded: Pakistan protests              - 70 months, max=100
  ✓ Loaded: Imran Khan                     - 70 months, max=100
  ✓ Loaded: Pakistan news                  - 70 months, max=100
  ✓ Loaded: Pakistan election              - 70 months, max=100.0
  ✓ Loaded: PTI Pakistan                   - 70 months, max=100
  ✓ Loaded: Pakistan political crisis      - 70 months, max=100


In [276]:
# Merge with ACLED data
merged = pakistan_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))


✓ Merged dataset: 70 months with 6 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Pakistan protests  Imran Khan  Pakistan news  Pakistan election  PTI Pakistan  Pakistan political crisis
2020-01-01     423          64                  3          14             11                1.0             5                          0
2020-02-01     531         126                  2          13             11                1.0             4                          0
2020-03-01     401          68                  1          17             21                0.5             3                          0
2020-04-01     503         112                  2          29             18                1.0             5                          0
2020-05-01     678          58                  2          16             16                1.0             4                          5
2020-06-01     886          80                  4          16             13                1.0           

In [277]:
# Correlations
 
print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")

correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

CORRELATION ANALYSIS: Search Terms vs ACLED Events

              Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
            Pakistan news        0.145899            0.453337           70
        Pakistan protests        0.132155            0.568285           70
        Pakistan election        0.063259            0.017526           70
             PTI Pakistan        0.055331            0.115212           70
Pakistan political crisis        0.014448            0.155708           70
               Imran Khan       -0.097533           -0.089341           70


In [278]:
 # Time-Lagged Correlation Analysis
 
print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
 
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")
print("-"*80)

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")


TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)
--------------------------------------------------------------------------------

Pakistan news:
  Lag -3 months (searches LEAD  ): correlation = +0.166
  Lag -2 months (searches LEAD  ): correlation = +0.156
  Lag -1 months (searches LEAD  ): correlation = +0.204
  Lag +0 months (CONCURRENT     ): correlation = +0.146
  Lag +1 months (searches LAG   ): correlation = +0.204
  Lag +2 months (searches LAG   ): correlation = +0.156
  Lag +3 months (searches LAG   ): correlation = +0.166

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Pakistan protests:
  Lag -3 months (searches LEAD  ): correlation = +0.188
  Lag -2 months (searches LEAD  ): correlation = +0.212
  Lag -1 months (searches LEAD  ): correlation = +0.231
  Lag +0 months (CONCURRENT     ): correlation = +0.132
  Lag +1 months (searche

In [279]:
#  Indentify Key Periods

print("KEY PERIODS")

# October 2020: Early spike
print("\n OCTOBER 2020 (Early Political Protests):")
oct_2020 = merged[merged['month'] == '2020-10-01']
if not oct_2020.empty:
    print(f"  ACLED Events: {oct_2020['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {oct_2020['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Oct 2020):")
    for term in trends_data.keys():
        if term in oct_2020.columns:
            val = oct_2020[term].values[0]
            if pd.notna(val):
                print(f"    - {term:30s}: {val:.0f}/100")

# March 2025: Recent spike
print("\n MARCH 2025 (Recent Escalation):")
mar_2025 = merged[merged['month'] == '2025-03-01']
if not mar_2025.empty:
    print(f"  ACLED Events: {mar_2025['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {mar_2025['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Mar 2025):")
    for term in trends_data.keys():
        if term in mar_2025.columns:
            val = mar_2025[term].values[0]
            if pd.notna(val):
                print(f"    - {term:30s}: {val:.0f}/100")

# April 2025: Sustained violence
print("\n APRIL 2025 (Sustained Period):")
apr_2025 = merged[merged['month'] == '2025-04-01']
if not apr_2025.empty:
    print(f"  ACLED Events: {apr_2025['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {apr_2025['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Apr 2025):")
    for term in trends_data.keys():
        if term in apr_2025.columns:
            val = apr_2025[term].values[0]
            if pd.notna(val):
                print(f"    - {term:30s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:30s}: {row[term]:.0f}/100")

# Top 5 fatality spikes
 
print("Top 5 Fatality Spikes:")
 
top_fatal = merged.nlargest(5, 'FATALITIES')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_fatal.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Fatality Rate: {(row['FATALITIES']/row['EVENTS']):.2f} per event")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:30s}: {row[term]:.0f}/100")

KEY PERIODS

 OCTOBER 2020 (Early Political Protests):
  ACLED Events: 1,380
  ACLED Fatalities: 71

  Google Trends (Oct 2020):
    - Pakistan protests             : 3/100
    - Imran Khan                    : 13/100
    - Pakistan news                 : 10/100
    - Pakistan election             : 1/100
    - PTI Pakistan                  : 5/100
    - Pakistan political crisis     : 6/100

 MARCH 2025 (Recent Escalation):
  ACLED Events: 1,236
  ACLED Fatalities: 523

  Google Trends (Mar 2025):
    - Pakistan protests             : 4/100
    - Imran Khan                    : 9/100
    - Pakistan news                 : 5/100
    - Pakistan election             : 1/100
    - PTI Pakistan                  : 4/100
    - Pakistan political crisis     : 7/100

 APRIL 2025 (Sustained Period):
  ACLED Events: 1,214
  ACLED Fatalities: 397

  Google Trends (Apr 2025):
    - Pakistan protests             : 6/100
    - Imran Khan                    : 9/100
    - Pakistan news                 

## VISUALIZATIONS

In [280]:
#  Visualizations
  
 
 

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Pakistan: Political Instability vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining political protests, terrorism, and PTI movement patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('pakistan_acled_vs_trends.html')
print("✓ Saved: pakistan_acled_vs_trends.html")

✓ Saved: pakistan_acled_vs_trends.html


In [281]:
chart

alt.Chart(...)

In [282]:
 
# Individual Comparison Charts
 
for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='darkred', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='darkred'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Pakistan: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"pakistan_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: pakistan_pakistan_news_comparison.html
✓ Saved: pakistan_pakistan_protests_comparison.html
✓ Saved: pakistan_pakistan_election_comparison.html


In [283]:
 
 # Event Type Anaylsis
 
print("EVENT TYPE BREAKDOWN")

# Get event types for the entire period
event_type_summary = pakistan_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Protests                      : 41,641 ( 77.6%)
  Battles                       :  3,893 (  7.3%)
  Violence against civilians    :  2,960 (  5.5%)
  Explosions/Remote violence    :  2,146 (  4.0%)
  Riots                         :  1,839 (  3.4%)
  Strategic developments        :  1,183 (  2.2%)


In [284]:
# Stats Summary
 
print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

# Compare early period (2020) vs recent (2025)
early = merged[merged['month'].dt.year == 2020]
recent = merged[merged['month'].dt.year == 2025]

 
print("TEMPORAL COMPARISON: 2020 vs 2025")
print("-"*80)

if len(early) > 0:
    print(f"\n2020:")
    print(f"  Avg Events/Month: {early['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {early['FATALITIES'].mean():.1f}")
    print(f"  Avg Fatality Rate: {early['fatality_rate'].mean():.3f}")

if len(recent) > 0:
    print(f"\n2025 (through October):")
    print(f"  Avg Events/Month: {recent['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {recent['FATALITIES'].mean():.1f}")
    print(f"  Avg Fatality Rate: {recent['fatality_rate'].mean():.3f}")

if len(early) > 0 and len(recent) > 0:
    change_events = ((recent['EVENTS'].mean() / early['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((recent['FATALITIES'].mean() / early['FATALITIES'].mean()) - 1) * 100
    print(f"\n2025 vs 2020: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 767
  Avg Fatalities/Month: 196.9
  Max Events (single month): 1,380 (October 2020)
  Max Fatalities (single month): 562 (September 2025)
  Avg Fatality Rate: 0.252 per event

Pearson Correlation Matrix:
                     EVENTS  FATALITIES  Pakistan news  Pakistan protests  Pakistan election
EVENTS             1.000000    0.498091       0.145899           0.132155           0.063259
FATALITIES         0.498091    1.000000       0.453337           0.568285           0.017526
Pakistan news      0.145899    0.453337       1.000000           0.625612           0.008360
Pakistan protests  0.132155    0.568285       0.625612           1.000000           0.007020
Pakistan election  0.063259    0.017526       0.008360           0.007020           1.000000
TEMPORAL COMPARISON: 2020 vs 2025
--------------------------------------------------------------------------------

2020:
  Avg Events/Month: 735


### Key Findings Summary
 
1. DUAL VIOLENCE PATTERN:
   - Political protests (PTI/Imran Khan movement)
   - Terrorism/insurgency (especially in border regions)
   - Mix creates complex attention dynamics

2. EVENT CHARACTERISTICS:
   - Protests dominate event counts
   - But terrorism drives fatalities
   - High variability across time periods

3. EXPECTED CORRELATION PATTERNS:
   - Weak correlations (r = 0.05-0.15 expected)
   - Political search terms may spike during crises
   - But overall attention limited (South Asia, complex politics)

4. IMRAN KHAN FACTOR:
   - Former PM as search anchor
   - Political imprisonment/trials
   - Tests if celebrity politician sustains attention

5. COMPARISON TO OTHER CASES:
   - Pakistan: Political protests + terrorism mix
   - India: Pure democratic protests (r = 0.08)
   - Myanmar: Coup → civil war (r = 0.32)
   - Similar to India but with higher violence

6. RECENT ESCALATION (2025):
   - March-May 2025 show high event counts
   - Tests if recent violence generates attention
   - Or if normalization has already occurred

## SYRIA

In [285]:
# Filter for Syria, 2020-2025
syria_acled = acled[
    (acled['COUNTRY'] == 'Syria') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
syria_acled['month'] = syria_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
syria_monthly = syria_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(syria_monthly)} months")
print(f"  Date range: {syria_monthly['month'].min()} to {syria_monthly['month'].max()}")
print(f"  Total events: {syria_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {syria_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = syria_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")

✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 72,821
  Total fatalities: 41,178

✓ Event type data loaded for detailed analysis


In [286]:
 
 # Load & Prepare Data

# Syria civil war trends files
trends_files = {
    'Syria civil war': 'Syria/syria_civil_war.csv',
    'Syria news': 'Syria/syria_news.csv',
    'Assad Syria': 'Syria.assad_syria.csv',
    'Aleppo': 'Syria/aleppo.csv',
    'Syrian conflict': 'Syria/syrian_conflict.csv',
    'Syria attack': 'Syria/syria_attack.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")

  ✓ Loaded: Syria civil war           - 70 months, max=100
  ✓ Loaded: Syria news                - 70 months, max=100
  ✗ Error loading Assad Syria: [Errno 2] No such file or directory: 'Syria.assad_syria.csv'
  ✓ Loaded: Aleppo                    - 70 months, max=100
  ✓ Loaded: Syrian conflict           - 70 months, max=100
  ✓ Loaded: Syria attack              - 70 months, max=100


In [287]:
 
 # 3. Merge ACLED & Trends Data

# Merge with ACLED data
merged = syria_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))

✓ Merged dataset: 70 months with 5 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Syria civil war  Syria news  Aleppo  Syrian conflict  Syria attack
2020-01-01    1311        1135               15          19      33               39            33
2020-02-01    1875        1579               33          41      38               51            31
2020-03-01     660         426               19          36      27               46            30
2020-04-01     715         510               10          14      31               30            20
2020-05-01     985         626               10          12      32               32            16
2020-06-01     792         492                8          12      33               28            16
2020-07-01     834         556                7          10      31               20            12
2020-08-01    1073         607                6          11      30               21            16
2020-09-01    1040         565       

In [288]:
 
# Correlations
 
print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")
 
correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))


CORRELATION ANALYSIS: Search Terms vs ACLED Events

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
Syria civil war        0.158112            0.335308           70
         Aleppo        0.088755            0.248720           70
Syrian conflict       -0.007970            0.260795           70
   Syria attack       -0.026697            0.070611           70
     Syria news       -0.093176            0.184545           70


In [289]:
# Time-Lagged Correlation Analysis
 
print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
 
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")
print("-"*80)

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)
--------------------------------------------------------------------------------

Syria civil war:
  Lag -3 months (searches LEAD  ): correlation = +0.106
  Lag -2 months (searches LEAD  ): correlation = +0.086
  Lag -1 months (searches LEAD  ): correlation = +0.159
  Lag +0 months (CONCURRENT     ): correlation = +0.158
  Lag +1 months (searches LAG   ): correlation = +0.159
  Lag +2 months (searches LAG   ): correlation = +0.086
  Lag +3 months (searches LAG   ): correlation = +0.106

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Aleppo:
  Lag -3 months (searches LEAD  ): correlation = +0.128
  Lag -2 months (searches LEAD  ): correlation = +0.116
  Lag -1 months (searches LEAD  ): correlation = +0.105
  Lag +0 months (CONCURRENT     ): correlation = +0.089
  Lag +1 months (searches LAG   )

In [ ]:
 
# 6.  Indentify Key Periods

print("KEY PERIODS")
 
# February 2020: Early high-intensity period
print("\n FEBRUARY 2020 (Early High-Intensity Period):")
feb_2020 = merged[merged['month'] == '2020-02-01']
if not feb_2020.empty:
    print(f"  ACLED Events: {feb_2020['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {feb_2020['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Feb 2020):")
    for term in trends_data.keys():
        if term in feb_2020.columns:
            val = feb_2020[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# November 2024: Recent major escalation
print("\n NOVEMBER 2024 (Recent Major Escalation):")
nov_2024 = merged[merged['month'] == '2024-11-01']
if not nov_2024.empty:
    print(f"  ACLED Events: {nov_2024['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {nov_2024['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Nov 2024):")
    for term in trends_data.keys():
        if term in nov_2024.columns:
            val = nov_2024[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# September 2023: Mid-period escalation
print("\n SEPTEMBER 2023 (Mid-Period Escalation):")
sep_2023 = merged[merged['month'] == '2023-09-01']
if not sep_2023.empty:
    print(f"  ACLED Events: {sep_2023['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {sep_2023['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Sep 2023):")
    for term in trends_data.keys():
        if term in sep_2023.columns:
            val = sep_2023[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# Top 5 fatality spikes
 
print("Top 5 Fatality Spikes:")
 
top_fatal = merged.nlargest(5, 'FATALITIES')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_fatal.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Fatality Rate: {(row['FATALITIES']/row['EVENTS']):.2f} per event")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

KEY PERIODS

📍 FEBRUARY 2020 (Early High-Intensity Period):
  ACLED Events: 1,875
  ACLED Fatalities: 1,579

  Google Trends (Feb 2020):
    - Syria civil war          : 33/100
    - Syria news               : 41/100
    - Aleppo                   : 38/100
    - Syrian conflict          : 51/100
    - Syria attack             : 31/100

📍 NOVEMBER 2024 (Recent Major Escalation):
  ACLED Events: 2,011
  ACLED Fatalities: 1,415

  Google Trends (Nov 2024):
    - Syria civil war          : 13/100
    - Syria news               : 6/100
    - Aleppo                   : 68/100
    - Syrian conflict          : 27/100
    - Syria attack             : 18/100

📍 SEPTEMBER 2023 (Mid-Period Escalation):
  ACLED Events: 1,684
  ACLED Fatalities: 838

  Google Trends (Sep 2023):
    - Syria civil war          : 5/100
    - Syria news               : 4/100
    - Aleppo                   : 24/100
    - Syrian conflict          : 17/100
    - Syria attack             : 8/100
Top 5 Event Spikes:

Novembe

## VISUALIZATIONS

In [ ]:
#  Visualizations

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Syria: Civil War Intensity vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining long-running civil war and attention fatigue patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
).interactive()

chart.save('syria_acled_vs_trends.html')
print("✓ Saved: syria_acled_vs_trends.html")

✓ Saved: syria_acled_vs_trends.html


In [292]:
chart

alt.Chart(...)

In [293]:
# Individual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='brown', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='brown'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Syria: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"syria_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: syria_syria_civil_war_comparison.html
✓ Saved: syria_aleppo_comparison.html
✓ Saved: syria_syrian_conflict_comparison.html


In [294]:
# Event Type Anaylsis
 
print("EVENT TYPE BREAKDOWN")
 

# Get event types for the entire period
event_type_summary = syria_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Explosions/Remote violence    : 33,944 ( 46.6%)
  Battles                       : 13,376 ( 18.4%)
  Violence against civilians    : 11,468 ( 15.7%)
  Strategic developments        :  8,820 ( 12.1%)
  Protests                      :  4,683 (  6.4%)
  Riots                         :    530 (  0.7%)


In [ ]:
# Stats Summary

print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")
print(f"  Median Fatality Rate: {merged['fatality_rate'].median():.3f} per event")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

# Temporal trends by year
 
print("TEMPORAL TRENDS (by Year)")
print("-"*80)

yearly_stats = merged.copy()
yearly_stats['year'] = yearly_stats['month'].dt.year
yearly_summary = yearly_stats.groupby('year').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()
yearly_summary['fatality_rate'] = yearly_summary['FATALITIES'] / yearly_summary['EVENTS']

print("\nYearly Summary:")
print(yearly_summary.to_string(index=False))

# Calculate coefficient of variation
events_cv = merged['EVENTS'].std() / merged['EVENTS'].mean()
fatalities_cv = merged['FATALITIES'].std() / merged['FATALITIES'].mean()

print(f"\nCoefficient of Variation:")
print(f"  Events:      {events_cv:.3f}")
print(f"  Fatalities:  {fatalities_cv:.3f}")

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 1040
  Avg Fatalities/Month: 588.3
  Max Events (single month): 2,011 (November 2024)
  Max Fatalities (single month): 2,684 (March 2025)
  Avg Fatality Rate: 0.565 per event
  Median Fatality Rate: 0.542 per event

Pearson Correlation Matrix:
                   EVENTS  FATALITIES  Syria civil war    Aleppo  Syrian conflict
EVENTS           1.000000    0.505608         0.158112  0.088755        -0.007970
FATALITIES       0.505608    1.000000         0.335308  0.248720         0.260795
Syria civil war  0.158112    0.335308         1.000000  0.780521         0.862082
Aleppo           0.088755    0.248720         0.780521  1.000000         0.761153
Syrian conflict -0.007970    0.260795         0.862082  0.761153         1.000000
TEMPORAL TRENDS (by Year)
--------------------------------------------------------------------------------

Yearly Summary:
 year  EVENTS  FATALITIES  fatality_rate
 2020  

### Key Findings Summary

1. THE "FORGOTTEN WAR" PATTERN:
   - Civil war ongoing since 2011 (14+ years)
   - Sustained high-intensity violence throughout 2020-2025
   - Extreme attention fatigue despite continued suffering

2. EVENT CHARACTERISTICS:
   - Explosions/remote violence dominate (~47%)
   - High fatality rate compared to protests
   - Sustained intensity with periodic escalations

3. EXPECTED CORRELATION PATTERNS:
   - Very weak correlations (r = 0.05-0.15 expected)
   - Complete normalization of violence
   - Only major escalations might generate brief attention

4. THE NOVEMBER 2024 ESCALATION TEST:
   - Major spike in events (2,011) and fatalities (1,415)
   - Tests if dramatic escalation can pierce attention fatigue
   - Critical test of "forgotten war" hypothesis

5. COMPARISON TO OTHER LONG-RUNNING CONFLICTS:
   - Syria: 14+ year civil war
   - Yemen: Similar "forgotten" pattern expected
   - Myanmar: Recent conflict (2021+), not yet "forgotten"
   - Ukraine: International war, sustained attention

6. THE COMPLETE INVISIBILITY HYPOTHESIS:
   - Long-running civil wars may reach zero correlation
   - Violence becomes completely "expected" and invisible
   - Geographic distance + complexity + duration = total neglect

## Yemen

In [296]:

# Filter for Yemen, 2020-2025
yemen_acled = acled[
    (acled['COUNTRY'] == 'Yemen') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
yemen_acled['month'] = yemen_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
yemen_monthly = yemen_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(yemen_monthly)} months")
print(f"  Date range: {yemen_monthly['month'].min()} to {yemen_monthly['month'].max()}")
print(f"  Total events: {yemen_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {yemen_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = yemen_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")

✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 58,708
  Total fatalities: 52,235

✓ Event type data loaded for detailed analysis


In [297]:
# Load & Prepare Data

# Yemen conflict trends files
trends_files = {
    'Yemen war': 'Yemen/yemen_war.csv',
    'Houthi': 'Yemen/houthi.csv',
    'Red Sea attacks': 'Yemen/red_sea_attacks.csv',
    'Houthi attacks': 'Yemen/houthi_attacks.csv',
    'Saudi Yemen': 'Yemen/saudi_yemen.csv',
    'Yemen': 'Yemen/yemen.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")


  ✗ Error loading Yemen war: [Errno 2] No such file or directory: 'Yemen/yemen_war.csv'
  ✓ Loaded: Houthi                    - 70 months, max=100
  ✓ Loaded: Red Sea attacks           - 70 months, max=100.0
  ✓ Loaded: Houthi attacks            - 70 months, max=100
  ✓ Loaded: Saudi Yemen               - 70 months, max=100
  ✓ Loaded: Yemen                     - 70 months, max=100


In [298]:
# 3. Merge ACLED & Trends Data

# Merge with ACLED data
merged = yemen_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))

✓ Merged dataset: 70 months with 5 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Houthi  Red Sea attacks  Houthi attacks  Saudi Yemen  Yemen
2020-01-01     910        1474       2              0.0               0           21     21
2020-02-01    1246        1851       1              0.0               0           21     21
2020-03-01    1076        1638       1              1.0               1           21     21
2020-04-01    1104        1970       1              0.0               0           27     29
2020-05-01    1076        1870       1              0.5               0           20     22
2020-06-01     914        1603       2              0.0               0           30     49
2020-07-01     851        1344       1              1.0               0           20     28
2020-08-01    1096        2002       1              1.0               0           15     22
2020-09-01     968        1752       1              0.0               0           17     21
2020-10-

In [299]:
 
# Correlations
 
print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")
 
correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

CORRELATION ANALYSIS: Search Terms vs ACLED Events

    Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
 Houthi attacks        0.242704           -0.310796           70
         Houthi        0.158711           -0.284860           70
          Yemen        0.126340           -0.270596           70
Red Sea attacks        0.093704           -0.326324           70
    Saudi Yemen        0.065344           -0.205495           70


In [300]:
 
# Time-Lagged Correlation Analysis
 
print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
 
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)

Houthi attacks:
  Lag -3 months (searches LEAD  ): correlation = +0.193
  Lag -2 months (searches LEAD  ): correlation = +0.294
  Lag -1 months (searches LEAD  ): correlation = +0.302
  Lag +0 months (CONCURRENT     ): correlation = +0.243
  Lag +1 months (searches LAG   ): correlation = +0.302
  Lag +2 months (searches LAG   ): correlation = +0.294
  Lag +3 months (searches LAG   ): correlation = +0.193

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Houthi:
  Lag -3 months (searches LEAD  ): correlation = +0.148
  Lag -2 months (searches LEAD  ): correlation = +0.163
  Lag -1 months (searches LEAD  ): correlation = +0.164
  Lag +0 months (CONCURRENT     ): correlation = +0.159
  Lag +1 months (searches LAG   ): correlation = +0.164
  Lag +2 months (searches LAG   ): correlation = +0.163
  L

In [301]:
 
# 6.  Indentify Key Periods
 
print("KEY PERIODS")
 
# 2024-2025: Red Sea attacks period (international attention)
print("\n JANUARY 2024 (Red Sea Escalation Begins):")
jan_2024 = merged[merged['month'] == '2024-01-01']
if not jan_2024.empty:
    print(f"  ACLED Events: {jan_2024['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {jan_2024['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Jan 2024):")
    for term in trends_data.keys():
        if term in jan_2024.columns:
            val = jan_2024[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n JULY 2024 (Peak Red Sea Period):")
jul_2024 = merged[merged['month'] == '2024-07-01']
if not jul_2024.empty:
    print(f"  ACLED Events: {jul_2024['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {jul_2024['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (Jul 2024):")
    for term in trends_data.keys():
        if term in jul_2024.columns:
            val = jul_2024[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

print("\n MAY 2025 (Recent Period):")
may_2025 = merged[merged['month'] == '2025-05-01']
if not may_2025.empty:
    print(f"  ACLED Events: {may_2025['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {may_2025['FATALITIES'].values[0]:,}")
    print(f"\n  Google Trends (May 2025):")
    for term in trends_data.keys():
        if term in may_2025.columns:
            val = may_2025[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

KEY PERIODS

 JANUARY 2024 (Red Sea Escalation Begins):
  ACLED Events: 705
  ACLED Fatalities: 210

  Google Trends (Jan 2024):
    - Houthi                   : 100/100
    - Red Sea attacks          : 100/100
    - Houthi attacks           : 90/100
    - Saudi Yemen              : 56/100
    - Yemen                    : 100/100

 JULY 2024 (Peak Red Sea Period):
  ACLED Events: 1,487
  ACLED Fatalities: 145

  Google Trends (Jul 2024):
    - Houthi                   : 13/100
    - Red Sea attacks          : 8/100
    - Houthi attacks           : 11/100
    - Saudi Yemen              : 18/100
    - Yemen                    : 33/100

 MAY 2025 (Recent Period):
  ACLED Events: 1,481
  ACLED Fatalities: 337

  Google Trends (May 2025):
    - Houthi                   : 17/100
    - Red Sea attacks          : 4/100
    - Houthi attacks           : 11/100
    - Saudi Yemen              : 21/100
    - Yemen                    : 35/100
Top 5 Event Spikes:

July 2024:
  ACLED Events: 1,487
  A

## VISUALIZATIONS

In [ ]:
#  Visualizations

# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Yemen: Conflict Intensity vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining civil war, Houthi insurgency, and Red Sea attacks',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Add annotation for Red Sea attacks period
redsea_date = pd.to_datetime('2024-01-01')
annotation_line = alt.Chart(pd.DataFrame({'date': [redsea_date]})).mark_rule(
    color='navy',
    strokeWidth=3,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [redsea_date],
    'y': [103],
    'label': ['↓ Red Sea Attacks Begin (2024)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='navy'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('yemen_acled_vs_trends.html')
print("✓ Saved: yemen_acled_vs_trends.html")

✓ Saved: yemen_acled_vs_trends.html


In [303]:
main_chart

alt.LayerChart(...)

In [304]:
# Individual Comparison Charts


for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='navy', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='navy'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Yemen: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"yemen_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: yemen_houthi_attacks_comparison.html
✓ Saved: yemen_houthi_comparison.html
✓ Saved: yemen_yemen_comparison.html


In [305]:
# Event Type Anaylsis
 
print("EVENT TYPE BREAKDOWN")

# Get event types for the entire period
event_type_summary = yemen_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Protests                      : 19,535 ( 33.3%)
  Explosions/Remote violence    : 17,151 ( 29.2%)
  Battles                       : 12,862 ( 21.9%)
  Strategic developments        :  4,867 (  8.3%)
  Violence against civilians    :  3,815 (  6.5%)
  Riots                         :    478 (  0.8%)


In [306]:
# Stats Summary

print("STATISTICAL SUMMARY")

# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

# Compare pre-Red Sea (2020-2023) vs Red Sea period (2024-2025)
pre_redsea = merged[merged['month'] < '2024-01-01']
redsea_period = merged[merged['month'] >= '2024-01-01']

 
print("COMPARISON: Pre-Red Sea (2020-2023) vs Red Sea Period (2024-2025)")

if len(pre_redsea) > 0:
    print(f"\nPre-Red Sea Period (2020-2023):")
    print(f"  Avg Events/Month: {pre_redsea['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {pre_redsea['FATALITIES'].mean():.1f}")

if len(redsea_period) > 0:
    print(f"\nRed Sea Period (2024-2025):")
    print(f"  Avg Events/Month: {redsea_period['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {redsea_period['FATALITIES'].mean():.1f}")

if len(pre_redsea) > 0 and len(redsea_period) > 0:
    change_events = ((redsea_period['EVENTS'].mean() / pre_redsea['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((redsea_period['FATALITIES'].mean() / pre_redsea['FATALITIES'].mean()) - 1) * 100
    print(f"\nRed Sea Period vs Pre: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%")

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 839
  Avg Fatalities/Month: 746.2
  Max Events (single month): 1,487 (July 2024)
  Max Fatalities (single month): 2,206 (February 2021)
  Avg Fatality Rate: 0.919 per event

Pearson Correlation Matrix:
                  EVENTS  FATALITIES  Houthi attacks    Houthi     Yemen
EVENTS          1.000000    0.139512        0.242704  0.158711  0.126340
FATALITIES      0.139512    1.000000       -0.310796 -0.284860 -0.270596
Houthi attacks  0.242704   -0.310796        1.000000  0.709370  0.684535
Houthi          0.158711   -0.284860        0.709370  1.000000  0.871693
Yemen           0.126340   -0.270596        0.684535  0.871693  1.000000
COMPARISON: Pre-Red Sea (2020-2023) vs Red Sea Period (2024-2025)

Pre-Red Sea Period (2020-2023):
  Avg Events/Month: 722
  Avg Fatalities/Month: 973.3

Red Sea Period (2024-2025):
  Avg Events/Month: 1093
  Avg Fatalities/Month: 250.8

Red Sea Period vs Pre: Events 

### Key Findings Summary
 
1. THE "FORGOTTEN WAR" WITH A TWIST:
   - 10+ year civil war (2015-2025)
   - Complete invisibility for internal conflict
   - But 2024 Red Sea attacks = international dimension
   - Tests if external threats can revive attention

2. DUAL CONFLICT PATTERN:
   - Internal civil war (Yemen itself): invisible
   - External attacks (Red Sea shipping): visible?
   - "Houthi" and "Red Sea attacks" may show spikes in 2024-2025
   - Tests if international impact overcomes duration fatigue

3. EXPECTED CORRELATION PATTERNS:
   - Overall: Very weak (r < 0.10, like Syria)
   - BUT: "Red Sea attacks" and "Houthi attacks" may show r = 0.15-0.25
   - Tests if EXTERNAL dimension creates attention when internal doesn't

4. THE RED SEA ESCALATION TEST (2024-2025):
   - Houthi attacks on commercial shipping
   - Direct threat to Western trade interests
   - US/UK military responses
   - Critical test: Does international impact > duration fatigue?

5. COMPARISON TO SYRIA:
   - Both: 10+ year civil wars
   - Both: Middle East location
   - Difference: Yemen has 2024 international dimension
   - Tests if Syria's zero correlation can be overcome by external threats

6. THE ULTIMATE ATTENTION HIERARCHY TEST:
   - Internal Yemeni suffering: Expected r < 0.05 (invisible)
   - External Red Sea attacks: Expected r = 0.15-0.25 (visible)
   - Same country, different attention based on Western impact

## Palestine

In [307]:
# 1. LOAD AND PREPARE ACLED DATA
 
# Filter for Palestine, 2020-2025
palestine_acled = acled[
    (acled['COUNTRY'] == 'Palestine') & 
    (acled['WEEK'] >= '2020-01-01')
].copy()

# Aggregate to monthly level
palestine_acled['month'] = palestine_acled['WEEK'].dt.to_period('M').dt.to_timestamp()
palestine_monthly = palestine_acled.groupby('month').agg({
    'EVENTS': 'sum',
    'FATALITIES': 'sum'
}).reset_index()

print(f"✓ ACLED Data: {len(palestine_monthly)} months")
print(f"  Date range: {palestine_monthly['month'].min()} to {palestine_monthly['month'].max()}")
print(f"  Total events: {palestine_monthly['EVENTS'].sum():,}")
print(f"  Total fatalities: {palestine_monthly['FATALITIES'].sum():,}")

# Also get event type breakdown
event_types = palestine_acled.groupby(['month', 'EVENT_TYPE'])['EVENTS'].sum().reset_index()
print(f"\n✓ Event type data loaded for detailed analysis")


✓ ACLED Data: 70 months
  Date range: 2020-01-01 00:00:00 to 2025-10-01 00:00:00
  Total events: 68,426
  Total fatalities: 70,366

✓ Event type data loaded for detailed analysis


In [308]:
# Load & Prepare Data

# Palestine/Gaza conflict trends files
trends_files = {
    'Gaza': 'Palestine CSVs/gaza.csv',
    'Israel Palestine': 'Palestine CSVs/israel palestine.csv',
    'Hamas Israel': 'Palestine CSVs/Hamas_Israel.csv',
    'Gaza war': 'Palestine CSVs/Gaza war.csv',
    'Israel Gaza attack': 'Palestine CSVs/israel_gaza_attack.csv',
    'Gaza strip': 'Palestine CSVs/gaza_strip.csv'
}

trends_data = {}

for name, filepath in trends_files.items():
    try:
        df = pd.read_csv(filepath, skiprows=1)
        df.columns = ['month', 'value']
        df['month'] = pd.to_datetime(df['month'])
        
        # Handle '<1' values
        df['value'] = df['value'].replace('<1', '0.5')
        df['value'] = pd.to_numeric(df['value'], errors='coerce')
        
        trends_data[name] = df
        print(f"  ✓ Loaded: {name:25s} - {len(df)} months, max={df['value'].max()}")
    except Exception as e:
        print(f"  ✗ Error loading {name}: {e}")

  ✓ Loaded: Gaza                      - 70 months, max=100
  ✓ Loaded: Israel Palestine          - 70 months, max=100
  ✓ Loaded: Hamas Israel              - 70 months, max=100.0
  ✓ Loaded: Gaza war                  - 70 months, max=100.0
  ✓ Loaded: Israel Gaza attack        - 70 months, max=100.0
  ✓ Loaded: Gaza strip                - 70 months, max=100.0


In [309]:
# 3. Merge ACLED & Trends Data
 
# Merge with ACLED data
merged = palestine_monthly.copy()

# Add each trends term
for name, df in trends_data.items():
    merged = merged.merge(
        df.rename(columns={'value': name}),
        on='month',
        how='left'
    )

print(f"✓ Merged dataset: {len(merged)} months with {len(trends_data)} search terms")

# Display sample of merged data
print("\nSample of merged data:")
sample_cols = ['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())
print(merged[sample_cols].head(10).to_string(index=False))

✓ Merged dataset: 70 months with 6 search terms

Sample of merged data:
     month  EVENTS  FATALITIES  Gaza  Israel Palestine  Hamas Israel  Gaza war  Israel Gaza attack  Gaza strip
2020-01-01     407           3     1                 2           0.5       0.5                 0.5         1.0
2020-02-01     643          11     2                 1           0.5       0.5                 0.5         1.0
2020-03-01     364           4     1                 1           0.5       0.5                 0.5         1.0
2020-04-01     341           2     2                 1           0.5       0.5                 0.5         1.0
2020-05-01     432           8     2                 1           0.5       0.5                 0.5         1.0
2020-06-01     328           2     1                 1           0.5       0.5                 0.5         1.0
2020-07-01     269           2     1                 2           0.5       0.5                 0.5         1.0
2020-08-01     440           9     2    

In [310]:
# Correlations
 
print("CORRELATION ANALYSIS: Search Terms vs ACLED Events")
 
correlations = []
for term in trends_data.keys():
    if term in merged.columns:
        valid_data = merged[['EVENTS', 'FATALITIES', term]].dropna()
        
        if len(valid_data) > 10:
            corr_events = valid_data['EVENTS'].corr(valid_data[term])
            corr_fatalities = valid_data['FATALITIES'].corr(valid_data[term])
            
            correlations.append({
                'Search Term': term,
                'Corr w/ Events': corr_events,
                'Corr w/ Fatalities': corr_fatalities,
                'Data Points': len(valid_data)
            })

corr_df = pd.DataFrame(correlations).sort_values('Corr w/ Events', ascending=False)
print("\n" + corr_df.to_string(index=False))

CORRELATION ANALYSIS: Search Terms vs ACLED Events

       Search Term  Corr w/ Events  Corr w/ Fatalities  Data Points
          Gaza war        0.765360            0.824808           70
              Gaza        0.709144            0.786207           70
Israel Gaza attack        0.599379            0.721554           70
  Israel Palestine        0.567191            0.728604           70
      Hamas Israel        0.535971            0.746333           70
        Gaza strip        0.470935            0.663626           70


In [311]:
# Time-Lagged Correlation Analysis
 
print("TIME-LAG ANALYSIS: Do searches LEAD or LAG events?")
 
print("Positive lag = Searches happen AFTER events (reactive)")
print("Negative lag = Searches happen BEFORE events (predictive)")
print("-"*80)

# Focus on top 3 search terms
top_terms = corr_df.head(3)['Search Term'].tolist()

for term in top_terms:
    print(f"\n{term}:")
     
    
    valid_data = merged[['EVENTS', term]].dropna()
    
    best_corr = -999
    best_lag = 0
    
    # Test lags from -3 to +3 months
    for lag in range(-3, 4):
        if lag == 0:
            corr = valid_data['EVENTS'].corr(valid_data[term])
        elif lag > 0:
            if len(valid_data) > lag:
                corr = valid_data['EVENTS'].iloc[lag:].corr(valid_data[term].iloc[:-lag])
            else:
                corr = 0
        else:
            if len(valid_data) > abs(lag):
                corr = valid_data['EVENTS'].iloc[:lag].corr(valid_data[term].iloc[-lag:])
            else:
                corr = 0
        
        if abs(corr) > abs(best_corr):
            best_corr = corr
            best_lag = lag
        
        direction = "searches LAG" if lag > 0 else ("searches LEAD" if lag < 0 else "CONCURRENT")
        print(f"  Lag {lag:+2d} months ({direction:15s}): correlation = {corr:+.3f}")
    
    interpretation = "REACTIVE (searches follow events)" if best_lag > 0 else \
                    "PREDICTIVE (searches precede events)" if best_lag < 0 else \
                    "CONCURRENT (searches match events)"
    print(f"\n  → Best correlation at lag {best_lag:+d}: {best_corr:+.3f} ({interpretation})")

TIME-LAG ANALYSIS: Do searches LEAD or LAG events?
Positive lag = Searches happen AFTER events (reactive)
Negative lag = Searches happen BEFORE events (predictive)
--------------------------------------------------------------------------------

Gaza war:
  Lag -3 months (searches LEAD  ): correlation = +0.820
  Lag -2 months (searches LEAD  ): correlation = +0.825
  Lag -1 months (searches LEAD  ): correlation = +0.820
  Lag +0 months (CONCURRENT     ): correlation = +0.765
  Lag +1 months (searches LAG   ): correlation = +0.820
  Lag +2 months (searches LAG   ): correlation = +0.825
  Lag +3 months (searches LAG   ): correlation = +0.820

  → Best correlation at lag +0: -999.000 (CONCURRENT (searches match events))

Gaza:
  Lag -3 months (searches LEAD  ): correlation = +0.727
  Lag -2 months (searches LEAD  ): correlation = +0.737
  Lag -1 months (searches LEAD  ): correlation = +0.738
  Lag +0 months (CONCURRENT     ): correlation = +0.709
  Lag +1 months (searches LAG   ): correla

In [312]:
 
# 6.  Indentify Key Periods

print("KEY PERIODS")
 
# October 2023: October 7 attacks and Gaza war begins
print("\n OCTOBER 2023 (October 7 Attacks - War Begins):")
oct_2023 = merged[merged['month'] == '2023-10-01']
if not oct_2023.empty:
    print(f"  ACLED Events: {oct_2023['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {oct_2023['FATALITIES'].values[0]:,}")
    print(f"  Fatality Rate: {(oct_2023['FATALITIES'].values[0]/oct_2023['EVENTS'].values[0]):.2f} per event")
    print(f"\n  Google Trends (Oct 2023):")
    for term in trends_data.keys():
        if term in oct_2023.columns:
            val = oct_2023[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# December 2023: Peak of initial war phase
print("\n DECEMBER 2023 (Peak War Intensity):")
dec_2023 = merged[merged['month'] == '2023-12-01']
if not dec_2023.empty:
    print(f"  ACLED Events: {dec_2023['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {dec_2023['FATALITIES'].values[0]:,}")
    print(f"  Fatality Rate: {(dec_2023['FATALITIES'].values[0]/dec_2023['EVENTS'].values[0]):.2f} per event")
    print(f"\n  Google Trends (Dec 2023):")
    for term in trends_data.keys():
        if term in dec_2023.columns:
            val = dec_2023[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# May 2025: Recent period
print("\n MAY 2025 (Recent Sustained Conflict):")
may_2025 = merged[merged['month'] == '2025-05-01']
if not may_2025.empty:
    print(f"  ACLED Events: {may_2025['EVENTS'].values[0]:,}")
    print(f"  ACLED Fatalities: {may_2025['FATALITIES'].values[0]:,}")
    print(f"  Fatality Rate: {(may_2025['FATALITIES'].values[0]/may_2025['EVENTS'].values[0]):.2f} per event")
    print(f"\n  Google Trends (May 2025):")
    for term in trends_data.keys():
        if term in may_2025.columns:
            val = may_2025[term].values[0]
            if pd.notna(val):
                print(f"    - {term:25s}: {val:.0f}/100")

# Top 5 event spikes
 
print("Top 5 Event Spikes:")
 
top_spikes = merged.nlargest(5, 'EVENTS')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_spikes.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Fatality Rate: {(row['FATALITIES']/row['EVENTS']):.2f} per event")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

# Top 5 fatality spikes
 
print("Top 5 Fatality Spikes:")
 
top_fatal = merged.nlargest(5, 'FATALITIES')[['month', 'EVENTS', 'FATALITIES'] + list(trends_data.keys())]
for idx, row in top_fatal.iterrows():
    print(f"\n{row['month'].strftime('%B %Y')}:")
    print(f"  ACLED Events: {row['EVENTS']:,}")
    print(f"  ACLED Fatalities: {row['FATALITIES']:,}")
    print(f"  Fatality Rate: {(row['FATALITIES']/row['EVENTS']):.2f} per event")
    print(f"  Search Interest:")
    for term in trends_data.keys():
        if pd.notna(row[term]):
            print(f"    - {term:25s}: {row[term]:.0f}/100")

KEY PERIODS

 OCTOBER 2023 (October 7 Attacks - War Begins):
  ACLED Events: 2,930
  ACLED Fatalities: 9,773
  Fatality Rate: 3.34 per event

  Google Trends (Oct 2023):
    - Gaza                     : 100/100
    - Israel Palestine         : 100/100
    - Hamas Israel             : 100/100
    - Gaza war                 : 100/100
    - Israel Gaza attack       : 100/100
    - Gaza strip               : 100/100

 DECEMBER 2023 (Peak War Intensity):
  ACLED Events: 2,525
  ACLED Fatalities: 7,616
  Fatality Rate: 3.02 per event

  Google Trends (Dec 2023):
    - Gaza                     : 23/100
    - Israel Palestine         : 13/100
    - Hamas Israel             : 12/100
    - Gaza war                 : 36/100
    - Israel Gaza attack       : 11/100
    - Gaza strip               : 8/100

 MAY 2025 (Recent Sustained Conflict):
  ACLED Events: 2,265
  ACLED Fatalities: 2,484
  Fatality Rate: 1.10 per event

  Google Trends (May 2025):
    - Gaza                     : 17/100
    - Isr

## VISUALIZATIONS

In [313]:
#  Visualizations
 
# Normalize data for comparison
merged_normalized = merged.copy()
merged_normalized['EVENTS_norm'] = (merged['EVENTS'] / merged['EVENTS'].max()) * 100
merged_normalized['FATALITIES_norm'] = (merged['FATALITIES'] / merged['FATALITIES'].max()) * 100

# Reshape for Altair
plot_data = []
for _, row in merged_normalized.iterrows():
    # Add ACLED data
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Events',
        'value': row['EVENTS_norm'],
        'type': 'Conflict Data',
        'raw_value': row['EVENTS']
    })
    plot_data.append({
        'month': row['month'],
        'metric': 'ACLED Fatalities',
        'value': row['FATALITIES_norm'],
        'type': 'Conflict Data',
        'raw_value': row['FATALITIES']
    })
    
    # Add top 3 search terms
    for term in top_terms:
        if term in row and pd.notna(row[term]):
            plot_data.append({
                'month': row['month'],
                'metric': f'Search: {term}',
                'value': row[term],
                'type': 'Google Trends',
                'raw_value': row[term]
            })

plot_df = pd.DataFrame(plot_data)

# Main time series chart
chart = alt.Chart(plot_df).mark_line(strokeWidth=2.5, point=True).encode(
    x=alt.X('month:T', 
            title='Month',
            axis=alt.Axis(format='%b %Y', labelAngle=-45)),
    y=alt.Y('value:Q', 
            title='Normalized Value (0-100)',
            scale=alt.Scale(domain=[0, 105])),
    color=alt.Color('metric:N', 
                    title='Metric',
                    scale=alt.Scale(scheme='tableau10')),
    strokeDash=alt.StrokeDash('type:N',
                               title='Data Type',
                               scale=alt.Scale(domain=['Conflict Data', 'Google Trends'],
                                             range=[[1,0], [5,3]])),
    tooltip=[
        alt.Tooltip('month:T', title='Month', format='%B %Y'),
        alt.Tooltip('metric:N', title='Metric'),
        alt.Tooltip('value:Q', title='Normalized', format='.1f'),
        alt.Tooltip('raw_value:Q', title='Raw Value', format=',.0f')
    ]
).properties(
    width=1400,
    height=450,
    title={
        'text': 'Palestine: Conflict Intensity vs Public Search Interest (2020-2025)',
        'subtitle': 'Examining October 7 attacks, Gaza war, and global attention patterns',
        'fontSize': 18,
        'subtitleFontSize': 13
    }
)

# Annotation for October 7, 2023
oct7_date = pd.to_datetime('2023-10-07')
annotation_line = alt.Chart(pd.DataFrame({'date': [oct7_date]})).mark_rule(
    color='red',
    strokeWidth=3,
    strokeDash=[5, 5]
).encode(x='date:T')

annotation_text = alt.Chart(pd.DataFrame({
    'date': [oct7_date],
    'y': [103],
    'label': ['↓ October 7 Attacks (2023)']
})).mark_text(
    align='center',
    baseline='bottom',
    fontSize=12,
    fontWeight='bold',
    color='red'
).encode(
    x='date:T',
    y=alt.Y('y:Q', scale=alt.Scale(domain=[0, 105])),
    text='label:N'
)

main_chart = chart + annotation_line + annotation_text

main_chart.save('palestine_acled_vs_trends.html')
print("✓ Saved: palestine_acled_vs_trends.html")

✓ Saved: palestine_acled_vs_trends.html


In [314]:
main_chart

alt.LayerChart(...)

In [315]:
# Individual Comparison Charts

for term in top_terms:
    term_data = merged[['month', 'EVENTS', 'FATALITIES', term]].dropna().copy()
    
    # Create dual-axis chart
    base = alt.Chart(term_data).encode(
        x=alt.X('month:T', 
                title='Month',
                axis=alt.Axis(format='%b %Y', labelAngle=-45))
    )
    
    # ACLED Events
    events_line = base.mark_line(color='steelblue', strokeWidth=3).encode(
        y=alt.Y('EVENTS:Q', 
                title='ACLED Events',
                axis=alt.Axis(titleColor='steelblue')),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    # Google Trends
    trends_line = base.mark_line(color='red', strokeWidth=3).encode(
        y=alt.Y(f'{term}:Q', 
                title=f'Google Trends: {term}',
                axis=alt.Axis(titleColor='red'),
                scale=alt.Scale(domain=[0, 100])),
        tooltip=[
            alt.Tooltip('month:T', title='Month', format='%B %Y'),
            alt.Tooltip('EVENTS:Q', title='Events', format=','),
            alt.Tooltip(f'{term}:Q', title='Search Interest', format='.0f')
        ]
    )
    
    term_chart = alt.layer(events_line, trends_line).resolve_scale(
        y='independent'
    ).properties(
        width=1200,
        height=400,
        title=f'Palestine: ACLED Events vs "{term}" Search Interest'
    ).interactive()
    
    filename = f"palestine_{term.lower().replace(' ', '_')}_comparison.html"
    term_chart.save(filename)
    print(f"✓ Saved: {filename}")

✓ Saved: palestine_gaza_war_comparison.html
✓ Saved: palestine_gaza_comparison.html
✓ Saved: palestine_israel_gaza_attack_comparison.html


In [316]:
# Event Type Anaylsis
 
print("EVENT TYPE BREAKDOWN")
 
# Get event types for the entire period
event_type_summary = palestine_acled.groupby('EVENT_TYPE')['EVENTS'].sum().sort_values(ascending=False)
print("\nTotal Events by Type (2020-2025):")
for event_type, count in event_type_summary.items():
    pct = (count / event_type_summary.sum()) * 100
    print(f"  {event_type:30s}: {count:6,} ({pct:5.1f}%)")

EVENT TYPE BREAKDOWN

Total Events by Type (2020-2025):
  Explosions/Remote violence    : 21,719 ( 31.7%)
  Riots                         : 21,445 ( 31.3%)
  Strategic developments        : 11,194 ( 16.4%)
  Battles                       :  5,437 (  7.9%)
  Violence against civilians    :  4,992 (  7.3%)
  Protests                      :  3,639 (  5.3%)


In [ ]:
# Stats Summary

print("STATISTICAL SUMMARY")
 
# Overall statistics
print("\nOverall Statistics (2020-2025):")
print(f"  Total Months: {len(merged)}")
print(f"  Avg Events/Month: {merged['EVENTS'].mean():.0f}")
print(f"  Avg Fatalities/Month: {merged['FATALITIES'].mean():.1f}")
print(f"  Max Events (single month): {merged['EVENTS'].max():,} ({merged.loc[merged['EVENTS'].idxmax(), 'month'].strftime('%B %Y')})")
print(f"  Max Fatalities (single month): {merged['FATALITIES'].max():,} ({merged.loc[merged['FATALITIES'].idxmax(), 'month'].strftime('%B %Y')})")

# Fatality rate
merged['fatality_rate'] = merged['FATALITIES'] / merged['EVENTS'].replace(0, np.nan)
print(f"  Avg Fatality Rate: {merged['fatality_rate'].mean():.3f} per event")
print(f"  Median Fatality Rate: {merged['fatality_rate'].median():.3f} per event")

# Correlation matrix
if len(top_terms) > 0:
    print("\nPearson Correlation Matrix:")
    corr_cols = ['EVENTS', 'FATALITIES'] + [term for term in top_terms if term in merged.columns]
    corr_matrix = merged[corr_cols].corr()
    print(corr_matrix.to_string())

# Compare pre-October 7 vs post-October 7
pre_oct7 = merged[merged['month'] < '2023-10-01']
post_oct7 = merged[merged['month'] >= '2023-10-01']

 
print("COMPARISON: Pre-October 7 (2020-Sep 2023) vs Post-October 7 (Oct 2023-2025)")

if len(pre_oct7) > 0:
    print(f"\nPre-October 7 Period:")
    print(f"  Avg Events/Month: {pre_oct7['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {pre_oct7['FATALITIES'].mean():.1f}")
    print(f"  Avg Fatality Rate: {pre_oct7['fatality_rate'].mean():.3f}")

if len(post_oct7) > 0:
    print(f"\nPost-October 7 Period:")
    print(f"  Avg Events/Month: {post_oct7['EVENTS'].mean():.0f}")
    print(f"  Avg Fatalities/Month: {post_oct7['FATALITIES'].mean():.1f}")
    print(f"  Avg Fatality Rate: {post_oct7['fatality_rate'].mean():.3f}")

if len(pre_oct7) > 0 and len(post_oct7) > 0:
    change_events = ((post_oct7['EVENTS'].mean() / pre_oct7['EVENTS'].mean()) - 1) * 100
    change_fatalities = ((post_oct7['FATALITIES'].mean() / pre_oct7['FATALITIES'].mean()) - 1) * 100
    change_fatality_rate = ((post_oct7['fatality_rate'].mean() / pre_oct7['fatality_rate'].mean()) - 1) * 100
    print(f"\nPost vs Pre: Events {change_events:+.1f}%, Fatalities {change_fatalities:+.1f}%, Fatality Rate {change_fatality_rate:+.1f}%")

# Attention decay analysis
 
print("ATTENTION DECAY ANALYSIS (Post-October 7)")

if not oct_2023.empty and len(top_terms) > 0:
    best_term = top_terms[0]
    
    oct_events = oct_2023['EVENTS'].values[0]
    oct_search = oct_2023[best_term].values[0] if best_term in oct_2023.columns else 0
    
    if not may_2025.empty:
        may_events = may_2025['EVENTS'].values[0]
        may_search = may_2025[best_term].values[0] if best_term in may_2025.columns else 0
        
        print(f"\nUsing '{best_term}':")
        print(f"Oct 2023 (War Begins): {oct_events:,} events → {oct_search:.0f}/100 search")
        print(f"May 2025 (Recent):     {may_events:,} events → {may_search:.0f}/100 search")
        
        if oct_search > 0 and may_search > 0:
            decay = ((may_search / oct_search) - 1) * 100
            print(f"\nAttention Change: {decay:+.1f}%")
            print(f"Despite ongoing conflict, attention {'increased' if decay > 0 else 'decreased'}")

STATISTICAL SUMMARY

Overall Statistics (2020-2025):
  Total Months: 70
  Avg Events/Month: 978
  Avg Fatalities/Month: 1005.2
  Max Events (single month): 2,930 (October 2023)
  Max Fatalities (single month): 9,773 (October 2023)
  Avg Fatality Rate: 0.530 per event
  Median Fatality Rate: 0.037 per event

Pearson Correlation Matrix:
                      EVENTS  FATALITIES  Gaza war      Gaza  Israel Gaza attack
EVENTS              1.000000    0.873639  0.765360  0.709144            0.599379
FATALITIES          0.873639    1.000000  0.824808  0.786207            0.721554
Gaza war            0.765360    0.824808  1.000000  0.965118            0.856999
Gaza                0.709144    0.786207  0.965118  1.000000            0.939846
Israel Gaza attack  0.599379    0.721554  0.856999  0.939846            1.000000
COMPARISON: Pre-October 7 (2020-Sep 2023) vs Post-October 7 (Oct 2023-2025)
--------------------------------------------------------------------------------

Pre-October 7 Perio

### Key Findings Summary

1. THE OCTOBER 7 MOMENT (Oct 2023):
   - 2,930 events, 9,773 fatalities (3.34 per event - EXTREME)
   - Highest fatality rate of ANY month in ANY country analyzed
   - Expected: Maximum search interest (100/100 across all terms)
   - Discrete, dramatic triggering event like Myanmar coup

2. SUSTAINED HIGH-INTENSITY WAR (Oct 2023 - Present):
   - Continued extremely high event counts and fatalities
   - Dec 2023: 2,525 events, 7,616 fatalities
   - May 2025: 2,265 events, 2,484 fatalities
   - Tests: Can attention sustain for 18+ months of war?

3. EXPECTED CORRELATION PATTERNS:
   - Strong initial correlation (r = 0.40-0.60 expected)
   - Similar to Myanmar coup or higher (discrete trigger)
   - But with ongoing sustained violence (like Ukraine)
   - Critical test of attention decay in high-profile conflicts

4. ATTENTION DECAY TEST:
   - Oct 2023: Expected 100/100 search across all terms
   - May 2025: How much has attention decayed after 19 months?
   - Comparison to Ukraine (Feb 2022 → Aug 2025 = 42 months)
   - Tests if Gaza maintains attention better/worse than Ukraine

5. GEOGRAPHIC AND POLITICAL FACTORS:
   - Middle East location (like Syria, Yemen)
   - BUT: Direct US ally involvement (Israel)
   - US domestic political salience (unlike Syria/Yemen)
   - Tests if Western political connection > geographic distance

6. THE FATALITY RATE EXTREME:
   - Oct 2023: 3.34 fatalities/event
   - Highest of ANY case analyzed
   - Ukraine (Aug 2025): 0.81/event
   - Myanmar (Jan 2022): 1.08/event
   - Tests: Do extreme fatalities correlate with extreme attention?

## Comparative Summary: All 10 Countries

This table synthesizes the complete analysis, ranking countries by correlation strength and revealing the systematic hierarchy of attention allocation.


## 4. Conclusions

### Core Finding: Attention Follows Political Salience, Not Human Suffering

Analysis of 884,216 conflict events from 2020-2025 across 11 countries reveals **systematic hierarchy** in attention allocation:

**The Hierarchy (by correlation strength):**
- **Tier 0 (r > 0.70):** Palestine (0.77), USA (0.75) - Domestic movements + geopolitical salience
- **Tier 1 (r = 0.30-0.50):** Ukraine (0.40), Myanmar (0.30) - International conflicts with major escalations
- **Tier 2 (r = 0.20-0.30):** Yemen Red Sea (0.24) - Western commercial interests
- **Tier 3 (r = 0.10-0.20):** Syria (0.16), Pakistan (0.15), Brazil (0.15) - Forgotten/electoral violence
- **Tier 4 (r < 0.10):** India (0.08), Mexico (0.08), Yemen Internal (0.03) - Normalized/forgotten

### Four Distinct Patterns Identified

**1. Sudden Shock (Ukraine, Myanmar, Palestine)**
- Exponential decay with ~3-month half-life
- Massive initial attention (100%) → 60-80% decline within 90 days
- Driven by dramatic triggering events (invasion, coup, October 7)

**2. Domestic Movements (USA - George Floyd protests)**
- Sustained attention through participatory engagement
- 496 deaths generate r = 0.75 (second-highest correlation)
- Demonstrates power of direct participation vs. distant observation

**3. Forgotten Wars (Syria, Yemen Internal)**
- Complete decoupling: violence continues, attention disappeared
- Syria: 41,178 deaths → r = 0.16
- Yemen: 52,235 deaths → r = 0.03
- Media fatigue, normalized as "permanent crises"

**4. Normalized Violence (Mexico)**
- 46,812 deaths framed as "crime" not "conflict" → r = 0.08
- Categorical framing determines visibility
- Death toll comparable to recognized wars, but invisible

### The Yemen Paradox: Quantified Proof

Yemen provides the **clearest evidence** that Western commercial interests, not humanitarian severity, drive attention:
- **Internal conflict:** 52,235 Yemeni deaths → r = 0.03 (effectively zero)
- **Red Sea attacks:** 0 Western deaths → r = 0.24 (8x more attention)

Same conflict, same actors, same region but 8-fold attention difference based solely on threats to Western shipping lanes.

### Death Toll Does Not Predict Attention

The scatter plot analysis (deaths vs. correlation) shows **no upward trend**. If attention followed suffering, higher death counts would correspond to higher correlations. Instead:

- Ukraine: 215,941 deaths → r = 0.40
- Mexico: 46,812 deaths → r = 0.08
- USA: 496 deaths → r = 0.75

This random pattern proves humanitarian severity does not predict attention allocation.

### Implications for Humanitarian Organizations

**Understanding predictable attention decay enables strategic planning:**

1. **Prepare for 3-month half-life:** Sudden shock crises lose 50% of attention within ~90 days regardless of escalation
2. **Build reserves during peaks:** Fundraising should maximize during attention spikes
3. **Redirect from forgotten wars:** Advocacy campaigns for Syria/Yemen may be ineffective; focus on diplomatic pressure
4. **Frame strategically:** How violence is categorized (conflict vs. crime) determines visibility
5. **Leverage domestic movements:** Participatory engagement sustains attention far longer than passive observation

**Key Insight for Advocacy:** Organizations cannot reverse attention decay through messaging alone—attention operates on structural constraints where it functions as a scarce resource with predictable allocation patterns based on Western political priorities.

### Research Context & Literature

These findings align with and quantify existing theories:
- **"Compassion Fatigue"** (Moeller, 1999): Documented through exponential decay curves
- **"Hierarchies of Suffering"** (Chouliaraki, 2006): Quantified via 25-fold correlation variation
- **"Accelerating Attention Dynamics"** (Lorenz-Spreen et al., 2019): Confirmed 3-month half-life
- **"Attention Economy"** (Simon, 1971): Validated through scarcity-driven allocation patterns

This analysis provides **empirical quantification** of previously theoretical concepts, enabling prediction and strategic response.

### Limitations & Future Research

**This analysis has important limitations:**

1. **Search as proxy:** Google Trends measures active information-seeking, not passive consumption (TV news, social media feeds)
2. **Western-centric:** English-language search terms may underrepresent Global South attention patterns
3. **Relative metrics:** Google Trends provides 0-100 normalized scores, not absolute search volumes
4. **Temporal scope:** Five-year window (2020-2025) may miss longer-term dynamics
5. **Correlation ≠ Causation:** High correlation indicates co-movement, not causal mechanism

**Future research directions:**
- Extend analysis to non-English search terms and non-Western countries
- Incorporate social media engagement metrics (Twitter, TikTok, Facebook)
- Analyze media coverage volume alongside search interest
- Investigate intervention strategies that might sustain attention
- Study how diplomatic outcomes correlate with attention patterns

### Final Reflection

This research demonstrates that **attention allocation is predictable, systematic, and largely independent of humanitarian need**. The "attention economy" of global conflict operates according to Western political salience, domestic proximity, and commercial interests rather than death tolls or human suffering.

Understanding these patterns doesn't change the underlying structural constraints, but it enables humanitarian organizations, journalists, and advocates to work more strategically within an attention-scarce environment. The goal is not to accept these inequalities but to understand them well enough to challenge them effectively.

**Key Takeaway:** Forgotten wars aren't forgotten by accident they're forgotten because attention operates as a scarce resource allocated according to power structures rather than humanitarian need. Quantifying these patterns is the first step toward changing them.